In [1]:
import math
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from scipy.optimize import fsolve
from scipy.integrate import quad
#  The function "equilibrium_hatalgebra" contained equilibrium conditions, which allows us to calculate the equilibrium price with given set of taxes
#  The function "minuswelfare" contained equations to calculate Home's welfare.
    ## The only difference between the two minuswelfare function is the fisrt argument input: I do this because scipy.minimize cosiders the first argument as the unkown to be solved by default.
#  The function "minimization" is used to maximize Home's welfare to get optimal tax values and energy price.
#  The function "callback_pe" takes the tax values calculated by welfare maximization to back out corresponding energy price.
#  The function "callback" takes optimal taxes and energy price to recalculate all other values including world emissions, home emissions...

In [361]:
def computejbar(ParaList, pe, te, varphi, tb_mat, tax_scenario, df):
    
    alpha, theta, sigma, sigmastar, epsilonD,epsilonDstar, epsilonS,epsilonSstar, beta, gamma, logit = ParaList
    
    ## optimal values
    # jxbar_hat =   (1 - df['jxbar']) ** (-1) / (((1 - df['jxbar']) ** (-1) - 1) + (1 + (1 - alpha) * tb_mat[0]/pe) ** (-theta) * (1 + tb_mat[0]/pe) ** ((1 - alpha) * theta));
    jxbar_hat = pe**(-alpha*theta) * (pe+tb_mat[0])**(-(1-alpha)*theta) / ( df['jxbar'] * pe**(-alpha*theta) * (pe+tb_mat[0])**(-(1-alpha)*theta) + (1-df['jxbar']) * (pe + (1-alpha) * tb_mat[0])**(-theta));
    j0_hat = (pe+tb_mat[0])**(-(1-alpha)*theta) / (df['jxbar'] * (pe+tb_mat[0])**(-(1-alpha)*theta) + (1-df['jxbar']) * pe**(-(1-alpha)*theta) );
    jmbar_hat = 1
    
    if tax_scenario['tax_sce']=='Unilateral':
        te=varphi;
        tb_mat[1]=1;
        
    if tax_scenario['tax_sce']=='purete':
        jxbar_hat = 1;   
        jmbar_hat = 1;
    
    if tax_scenario['tax_sce']=='puretc':
        te=tb_mat[0];
        jxbar_hat = 1;
        jmbar_hat = 1;
        tb_mat[1]=1;
    
    if tax_scenario['tax_sce']=='puretp':
        te=tb_mat[0];
        jxbar_hat =  (1 - df['jxbar']) ** (-1) / ((1 - df['jxbar']) ** (-1) - 1 + (1 + tb_mat[0]/pe) ** (theta * (1 - alpha)));
        jmbar_hat = (1 + ((1 - df['jmbar']) ** (-1) - 1) ** (-1)) / (1 + ((1 - df['jmbar']) ** (-1) - 1) ** (-1) * (1 + tb_mat[0]/pe) ** (theta * (1 - alpha)));
        tb_mat[1]=1;

    if tax_scenario['tax_sce']=='EC_hybrid':
        te=varphi;
        jxbar_hat = 1;
        jmbar_hat = 1;
        tb_mat[1]=1;

    if tax_scenario['tax_sce']=='PC_hybrid':
        te=tb_mat[0];
        if tb_mat[1]<0:
            tb_mat[1]=0
        jxbar_hat =  (1 - df['jxbar']) ** (-1) / ((1 - df['jxbar']) ** (-1) - 1 + (1 + (tb_mat[0] - tb_mat[2] * tb_mat[0])/pe) ** (theta * (1 - alpha)));
        jmbar_hat = (1 + ((1 - df['jmbar']) ** (-1) - 1) ** (-1)) / (1 + ((1 - df['jmbar']) ** (-1) - 1) ** (-1) * ((pe + tb_mat[0])/(pe + tb_mat[1] * tb_mat[0])) ** (theta * (1 - alpha)));
    
    if tax_scenario['tax_sce']=='EP_hybrid':
        te=tb_mat[1];
        jxbar_hat =  (1 - df['jxbar']) ** (-1) / ((1 - df['jxbar']) ** (-1) - 1 + (1 + tb_mat[0]/pe) ** (theta * (1 - alpha)));
        jmbar_hat = (1 + ((1 - df['jmbar']) ** (-1) - 1) ** (-1)) / (1 + ((1 - df['jmbar']) ** (-1) - 1) ** (-1) * (1 + tb_mat[0]/pe) ** (theta * (1 - alpha)));

    if tax_scenario['tax_sce']=='EPC_hybrid':
        te=varphi;
        if tb_mat[1]<0:
            tb_mat[1]=0
        jxbar_hat =  (1 - df['jxbar']) ** (-1) / ((1 - df['jxbar']) ** (-1) - 1 + (1 + (tb_mat[0] - tb_mat[2] * tb_mat[0])/pe) ** (theta * (1 - alpha)));
        jmbar_hat = (1 + ((1 - df['jmbar']) ** (-1) - 1) ** (-1)) / (1 + ((1 - df['jmbar']) ** (-1) - 1) ** (-1) * ((pe + tb_mat[0])/(pe + tb_mat[1] * tb_mat[0])) ** (theta * (1 - alpha)))
        
    return (te, jxbar_hat, jmbar_hat, j0_hat, tb_mat)

## compute values for the incomplete beta functions
def imcomp_betas(j0_prime, jxbar_prime, theta, sigmastar):
    def tempFunction(i, theta, sigmastar):
        return (i ** ((1 + theta) / theta - 1) * (1 - i) ** ((theta - sigmastar) / theta - 1)) 
    
    Bfunvec1_prime = quad(tempFunction,0,j0_prime, args=(theta, sigmastar))[0]
    Bfunvec2_prime = quad(tempFunction,0,jxbar_prime, args=(theta, sigmastar))[0]
    
    return (Bfunvec1_prime, Bfunvec2_prime)
    
## computes extraction values (home, foreign)
def compute_qe(petbte, epsilonS, epsilonSstar, logit, beta, gamma, pe, df):
    if logit==1:
        epsilonS=beta*(1-gamma)/(1-gamma+gamma*petbte**beta)
        epsilonSstar=beta*(1-gamma)/(1-gamma+gamma*pe**beta)
        Qe_hat = (petbte)**beta/(1-gamma+gamma*(petbte)**beta)
        Qestar_hat = pe**beta/(1-gamma+gamma*pe**beta)
        
    ## compute hat values    
    Qe_hat = (petbte) ** epsilonS
    Qestar_hat = pe ** epsilonSstar
    
    ## compute final values
    Qe_prime = df['Qe'] * Qe_hat
    Qestar_prime = df['Qestar'] * Qestar_hat
    
    return (Qe_prime, Qestar_prime)


## computes consumption values (home, import, export, foreign)
def compute_ce(pe, tb_mat,j0_hat, j0_prime, jmbar_hat, jmbar_prime,jxbar_hat, jxbar_prime, ParaList, df, tax_scenario):
    alpha, theta, sigma, sigmastar, epsilonD,epsilonDstar, epsilonS,epsilonSstar, beta, gamma, logit = ParaList
    
    Bfunvec1_prime, Bfunvec2_prime = imcomp_betas(j0_prime, jxbar_prime, theta, sigmastar)
    
    CeHH_hat = (pe + tb_mat[0]) ** (-epsilonD) * jmbar_hat ** (1 + (1 - sigma)/theta);
    CeHH_prime = df['CeHH'] * CeHH_hat;
       
    
    # CeFH_hat = (1 + (1 - sigmastar)/theta) * pe ** (-(1 - alpha) * sigmastar) * (pe + tb_mat[0]) ** (-alpha) * Bfunvec_prime/(df['jxbar'] ** (1 +1/theta)) * (1 - df['jxbar']) ** (sigmastar/theta);
    CeFH1_hat = (pe +tb_mat[0])**(-epsilonDstar) * j0_hat**(1 + (1 - sigmastar)/theta);
    CeFH2_hat = (1 + (1 - sigmastar)/theta) * ((1-df['jxbar'])/df['jxbar'])**(sigmastar/theta) * pe**(-epsilonDstar) * (1 + tb_mat[0]/pe)**(-alpha) * (Bfunvec2_prime - Bfunvec1_prime)/df['jxbar']**(1+(1-sigmastar)/theta);
    CeFH1_prime = df['CeFH'] * CeFH1_hat;
    CeFH2_prime = df['CeFH'] * CeFH2_hat;
    CeFH_hat = CeFH1_hat + CeFH2_hat;
    if tax_scenario['Base']==1:
        CeFH_hat = pe ** (-epsilonDstar) * jxbar_hat ** (1 + (1 - sigmastar)/theta);
    
    if tax_scenario['tax_sce']=='puretp' or tax_scenario['tax_sce']=='EP_hybrid':
        CeFH_hat = (pe + tb_mat[0]) ** (-epsilonDstar) * jxbar_hat ** (1 + (1 - sigmastar)/theta);
    
    if tax_scenario['tax_sce']=='PC_hybrid' or tax_scenario['tax_sce']=='EPC_hybrid':
        CeFH_hat = (pe + tb_mat[0] - tb_mat[2] * tb_mat[0]) ** (-epsilonDstar) * jxbar_hat ** (1 + (1 - sigmastar)/theta);
    
    if np.isnan(CeFH_hat)==True:
        CeFH_hat=0
    CeFH_prime =df['CeFH'] * CeFH_hat;
    
    
    CeHF_hat = (pe + tb_mat[0]) ** (-epsilonD);
    if tax_scenario['tax_sce']=='puretp' or tax_scenario['tax_sce']=='EP_hybrid':
        CeHF_hat = (pe) ** (-epsilonD) * ((1 - jmbar_prime)/(1 - df['jmbar'])) ** (1 + (1 - sigma)/theta);
    
    if tax_scenario['tax_sce']=='PC_hybrid' or tax_scenario['tax_sce']=='EPC_hybrid':
        CeHF_hat = (pe + tb_mat[1] * tb_mat[0]) ** (-epsilonD) * ((1 - jmbar_prime)/(1 - df['jmbar'])) ** (1 + (1 - sigma)/theta);
    
    CeHF_prime = df['CeHF'] * CeHF_hat;
    
    
    CeFF_prime = df['CeFF'] * ((1 - jxbar_prime)/(1-df['jxbar'])) ** (1 + (1 - sigmastar)/theta) * pe ** (-epsilonDstar);
    
    return (CeHH_prime, CeFH1_prime, CeFH2_prime, CeFH_prime, CeHF_prime, CeFF_prime, CeFH_hat, CeFH1_hat, CeHF_hat)



def comp_vg(pe, tb_mat, CeFH1_hat, j0_prime, jxbar_prime, CeFH_hat, CeHF_hat, df, tax_scenario, ParaList):
    
    alpha, theta, sigma, sigmastar, epsilonD,epsilonDstar, epsilonS,epsilonSstar, beta, gamma, logit = ParaList
    
    ##
    VgHH = df['CeHH']/(1 - alpha)
    VgFF = df['CeFF']/(1 - alpha)
    
    VgFH = df['CeFH'] /(1 - alpha)
    # VgFH_prime = VgFH * pe ** ((1 - sigmastar) * (1 - alpha)) * (1 - (1 - jxbar_prime) ** (1 + (1 - sigmastar)/theta))/ (df['jxbar'] * (1 - df['jxbar']) ** ( (1-sigmastar)/theta))
    VgFH1_hat = (pe + tb_mat[0]) * CeFH1_hat
    VgFH2_hat = pe**(1 - epsilonDstar) * ((1-j0_prime)**(1+(1-sigmastar)/theta) - (1-jxbar_prime)**(1+(1-sigmastar)/theta))/ (df['jxbar']  * (1 - df['jxbar'] )**( (1-sigmastar)/theta))
    VgFH1_prime = VgFH * VgFH1_hat
    VgFH2_prime = VgFH * VgFH2_hat
    VgFH_hat = VgFH1_hat + VgFH2_hat
    
    if tax_scenario['Base']==1:
        VgFH_hat = pe * CeFH_hat
        
    if tax_scenario['tax_sce']=='puretp' or tax_scenario['tax_sce']=='EP_hybrid':
        VgFH_hat = (pe + tb_mat[0]) * CeFH_hat
        
    if tax_scenario['tax_sce']=='PC_hybrid' or tax_scenario['tax_sce']=='EPC_hybrid':
        VgFH_hat = (pe + tb_mat[0] - tb_mat[2] * tb_mat[0]) * CeFH_hat
        
    if np.isnan(VgFH_hat)==True:
            VgFH_hat=0
    VgFH_prime = VgFH * VgFH_hat

    VgHF = df['CeHF']/(1 - alpha)
    VgHF_hat = (pe + alpha * tb_mat[0]) * CeHF_hat
    
    if tax_scenario['tax_sce']=='puretp' or tax_scenario['tax_sce']=='EP_hybrid':
        VgHF_hat = pe * CeHF_hat
        
    if tax_scenario['tax_sce']=='PC_hybrid' or tax_scenario['tax_sce']=='EPC_hybrid':
        VgHF_hat = (pe + alpha * tb_mat[1] * tb_mat[0]) * CeHF_hat
        
    VgHF_prime = VgHF * VgHF_hat
    
    return (VgHH, VgFF, VgFH1_prime, VgFH2_prime, VgFH_prime, VgHF_prime, VgFH, VgHF)

def comp_ve(pe, tb_mat, Ce_prime, tax_scenario, CeHH_prime, CeHF_prime, CeFH_prime, CeFH1_prime, CeFH2_prime, CeFF_prime, Cestar_prime):
    Ve_prime=(pe+tb_mat[0]) * Ce_prime
    if tax_scenario['tax_sce']=='puretp' or tax_scenario['tax_sce']=='EP_hybrid':
        Ve_prime = (pe+tb_mat[0]) * CeHH_prime + pe * CeHF_prime
        
    if tax_scenario['tax_sce']=='PC_hybrid' or tax_scenario['tax_sce']=='EPC_hybrid':
        Ve_prime = (pe+tb_mat[0]) * CeHH_prime + (pe + tb_mat[1]*tb_mat[0]) * CeHF_prime
    
    Vestar_prime= (pe+tb_mat[0]) * CeFH_prime  + pe * CeFF_prime
    Vestar_prime= (pe+tb_mat[0]) * CeFH1_prime + pe * CeFH2_prime + pe * CeFF_prime
    
    if tax_scenario['tax_sce']=='puretc' or tax_scenario['tax_sce']=='EC_hybrid' :
        Vestar_prime = pe * Cestar_prime
        
    if tax_scenario['tax_sce']=='PC_hybrid' or tax_scenario['tax_sce']=='EPC_hybrid':
        Vestar_prime = (pe + tb_mat[0] - tb_mat[2]*tb_mat[0]) * CeFH_prime + pe * CeFF_prime
        
    return (Ve_prime, Vestar_prime)

def comp_vgfin(pe, tb_mat, Ce_hat, Cestar_prime, CeHH_prime, CeHF_prime, CeFH_prime, CeFF_prime,VgFH_prime, ParaList, df, tax_scenario):
    alpha, theta, sigma, sigmastar, epsilonD,epsilonDstar, epsilonS,epsilonSstar, beta, gamma, logit = ParaList
    
    Vg = df['Ce'] /(1-alpha)
    Vg_prime_hat = (pe + tb_mat[0]) * Ce_hat
    Vg_prime = Vg * Vg_prime_hat
    if tax_scenario['tax_sce']=='puretp' or tax_scenario['tax_sce']=='EP_hybrid':
        Vg_prime = CeHH_prime/(1-alpha) * (pe + tb_mat[0]) + CeHF_prime/(1-alpha) * pe
        
    if tax_scenario['tax_sce']=='PC_hybrid' or tax_scenario['tax_sce']=='EPC_hybrid':
        Vg_prime = CeHH_prime/(1-alpha) * (pe + tb_mat[0]) + CeHF_prime/(1-alpha) * (pe + tb_mat[1] * tb_mat[0]);   
    
    Vgstar = df['Cestar'] /(1-alpha)
    Vgstar_prime = VgFH_prime + CeFF_prime/(1-alpha)* pe
    if tax_scenario['tax_sce']=='puretc' or tax_scenario['tax_sce']=='EC_hybrid' :
        Vgstar_prime = Cestar_prime/(1-alpha)* pe
        
    if tax_scenario['tax_sce']=='PC_hybrid' or tax_scenario['tax_sce']=='EPC_hybrid':
        Vgstar_prime = CeFF_prime/(1-alpha)* pe + CeFH_prime/(1-alpha)* (pe + tb_mat[0] - tb_mat[2]*tb_mat[0])
    
    return (Vg, Vg_prime, Vgstar, Vgstar_prime)

def comp_lg(pe, tb_mat, VgFH_prime, Ce_hat, Ge_prime, Gestar_prime, CeHH_prime, CeFH_prime, CeHF_prime, CeFF_prime, ParaList, df, tax_scenario):
    alpha, theta, sigma, sigmastar, epsilonD,epsilonDstar, epsilonS,epsilonSstar, beta, gamma, logit = ParaList
    
    Lg = alpha/(1-alpha) * df['Ge']
    Lg_prime = alpha/(1-alpha) * (pe + tb_mat[0]) * Ge_prime
    if tax_scenario['tax_sce']=='puretc' or tax_scenario['tax_sce']=='EC_hybrid':
        Lg_prime = alpha/(1-alpha) * (pe + tb_mat[0]) * CeHH_prime + alpha/(1-alpha) * pe * CeFH_prime
        
    if tax_scenario['tax_sce']=='PC_hybrid' or tax_scenario['tax_sce']=='EPC_hybrid':
        Lg_prime = alpha/(1-alpha) * (pe + tb_mat[0]) * CeHH_prime + alpha/(1-alpha) * (pe + tb_mat[0] - tb_mat[2] * tb_mat[0]) * CeFH_prime    
    
    Lgstar = alpha/(1-alpha) * df['Gestar']
    Lgstar_prime = alpha/(1-alpha) * (pe+tb_mat[0]) * CeHF_prime +alpha/(1-alpha) * pe * CeFF_prime
    if tax_scenario['tax_sce']=='puretp' or tax_scenario['tax_sce']=='EP_hybrid':
        Lgstar_prime = alpha/(1-alpha) * pe * Gestar_prime
    if tax_scenario['tax_sce']=='PC_hybrid' or tax_scenario['tax_sce']=='EPC_hybrid':
        Lgstar_prime = alpha/(1-alpha) * (pe + tb_mat[1]*tb_mat[0]) * CeHF_prime + alpha/(1-alpha) * pe * CeFF_prime
        
    return (Lg, Lg_prime, Lgstar, Lgstar_prime)

def comp_delta(Lg, Lg_prime, Lgstar, Lgstar_prime, Qeworld_prime, Vg, Vgstar, df, j0_prime, jmbar_prime, jxbar_prime, pe, petbte, tb_mat, tax_scenario, varphi, ParaList):
    
    alpha, theta, sigma, sigmastar, epsilonD,epsilonDstar, epsilonS,epsilonSstar, beta, gamma, logit = ParaList
    
    if pe < 0:
        pe = 0.0001
    
    delta_Le = (epsilonS/(epsilonS + 1)) * df['Qe'] * (petbte**(epsilonS + 1) - 1);
    delta_Lestar = (epsilonSstar/(epsilonSstar + 1)) * df['Qestar'] * (pe**(epsilonSstar + 1) - 1);
    
    def Func(a, beta, gamma):
        return (((1-gamma)*a**beta)/(1-gamma+gamma*a**beta)**2)
    
    if logit==1:
        delta_Le = beta * df['Qe'] * quad(Func,1,petbte, args=(beta, gamma))[0];
        delta_Lestar = beta * df['Qestar'] * quad(Func,1,pe, args=(beta, gamma))[0];
       
    delta_U = -delta_Le - delta_Lestar - (Lg_prime - Lg) - (Lgstar_prime - Lgstar) \
           + Vg * (alpha - 1) * math.log(pe + tb_mat[0]) + Vgstar * (1/theta) * math.log(df['jxbar']/j0_prime * (pe+tb_mat[0])**(-(1-alpha)*theta)) \
           - varphi * (Qeworld_prime - df['Qeworld']);

    if tax_scenario['tax_sce']=='puretc' or tax_scenario['tax_sce']=='purete' or tax_scenario['tax_sce']=='EC_hybrid':
        delta_U = -delta_Le - delta_Lestar - (Lg_prime - Lg) - (Lgstar_prime - Lgstar) \
           + Vg * (alpha - 1) * math.log(pe +  tb_mat[0]) + Vgstar * (alpha - 1) * math.log(pe) \
           - varphi * (Qeworld_prime - df['Qeworld']);

    if tax_scenario['tax_sce']=='puretp' or tax_scenario['tax_sce']=='EP_hybrid':
        delta_U = -delta_Le - delta_Lestar - (Lg_prime - Lg) - (Lgstar_prime - Lgstar) \
               +  Vg * ((alpha - 1) * math.log(pe + tb_mat[0]) + 1/theta * math.log(df['jmbar']/jmbar_prime)) \
               +  Vgstar * ((alpha - 1) * math.log(pe + tb_mat[0]) + 1/theta * math.log(df['jxbar']/jxbar_prime)) \
               - varphi * (Qeworld_prime - df['Qeworld']);
    
    if tax_scenario['tax_sce']=='PC_hybrid' or tax_scenario['tax_sce']=='EPC_hybrid':
        delta_U = -delta_Le - delta_Lestar - (Lg_prime - Lg) - (Lgstar_prime - Lgstar) \
               +  Vg * ((alpha - 1) * math.log(pe + tb_mat[0]) + 1/theta * math.log(df['jmbar']/jmbar_prime)) \
               +  Vgstar * ((alpha - 1) * math.log(pe + tb_mat[0] - tb_mat[2] * tb_mat[0]) + 1/theta * math.log(df['jxbar']/jxbar_prime)) \
               - varphi * (Qeworld_prime - df['Qeworld']);
    return delta_Le, delta_Lestar, delta_U, pe

def comp_leak(Qestar_prime, Gestar_prime, Cestar_prime, Qeworld_prime, df):
    
    leakage1 = -(Qestar_prime - df['Qestar'])/(Qeworld_prime - df['Qeworld']);
    leakage2 = -(Gestar_prime - df['Gestar'])/(Qeworld_prime - df['Qeworld']);
    leakage3 = -(Cestar_prime - df['Cestar'])/(Qeworld_prime - df['Qeworld']);   
    
    return (leakage1, leakage2, leakage3)
    
    

In [202]:
def equilibrium_hatalgebra(pe,*data):
    tb_mat, te, varphi, tax_scenario, ParaList, df = data
    alpha, theta, sigma, sigmastar, epsilonD,epsilonDstar, epsilonS,epsilonSstar, beta, gamma, logit = ParaList

    ## compute extraction tax, and jbar's
    te, jxbar_hat, jmbar_hat, j0_hat, tb_mat = computejbar(ParaList, pe, te, varphi, tb_mat, tax_scenario, df)

    jxbar_prime = jxbar_hat * df['jxbar']
    jmbar_prime = jmbar_hat * df['jmbar']
    j0_prime = j0_hat * df['jxbar']

    #if te is too large, HoGe stop producing
    petbte = pe + tb_mat[0] - te
    z = pe + tb_mat[0] >= te
    petbte = petbte * z
    
    ## compute home and foreign extractions
    Qe_prime, Qestar_prime = compute_qe(petbte, epsilonS, epsilonSstar, logit, beta, gamma, pe, df)

    ## compute consumptions around the world
    CeHH_prime, CeFH1_prime, CeFH2_prime, CeFH_prime, CeHF_prime, CeFF_prime, CeFH_hat, CeFH1_hat, CeHF_hat = compute_ce(pe, tb_mat,j0_hat, j0_prime, jmbar_hat, jmbar_prime,jxbar_hat, jxbar_prime, ParaList, df, tax_scenario)
    
    diff = Qe_prime + Qestar_prime - (CeHH_prime + CeFH_prime + CeHF_prime + CeFF_prime)
    #print('diff='+str(diff))
    return diff

In [200]:
def minuswelfare(tb_mat, te, varphi, tax_scenario, ParaList, df):
     #solve for equilibrium
    data = (tb_mat, te, varphi, tax_scenario, ParaList, df)
    pe = fsolve(equilibrium_hatalgebra,1,args=data, full_output = True)
    #if (pe[2] != 1):
    #    print("pe is", pe)
    #    print("tb is", tb_mat, "te is", te)
    #    print("tax is", tax_scenario['tax_sce'])
    #    print("opt value is", equilibrium_hatalgebra(pe[0][0], *data))
    pe=pe[0][0]
    # print('pe='+str(pe))
    if pe-te<0 and tax_scenario['tax_sce']=='purete':
        pe=te;
        
    # if pe+tb_mat[0]-tb_mat[1]<=0 and tax_scenario['tax_sce']=='EP_hybrid':
    #     # print(tb_mat)
    #     # print(pe)
    #     # tb_mat[0]=tb_mat[1]-pe;
    #     tb_mat[1]=pe+tb_mat[0];
    #     te=tb_mat[1];
 
    alpha, theta, sigma, sigmastar, epsilonD,epsilonDstar, epsilonS, epsilonSstar, beta, gamma, logit = ParaList
    
    ## compute extraction tax, and jbar_hat's
    te, jxbar_hat, jmbar_hat, j0_hat, tb_mat = computejbar(ParaList, pe, te, varphi, tb_mat, tax_scenario, df)
    
    jxbar_prime = jxbar_hat * df['jxbar'];
    jmbar_prime = jmbar_hat * df['jmbar'];
    j0_prime = j0_hat * df['jxbar'];

    #if te is too large, Home stop producing
    petbte = pe + tb_mat[0] - te;
    z = pe + tb_mat[0] >= te;
    petbte = petbte * z;
    
    ## home and foreign extractions
    Qe_prime, Qestar_prime = compute_qe(petbte, epsilonS, epsilonSstar, logit, beta, gamma, pe, df)
    
    ## compute consumptions around the world
    CeHH_prime, CeFH1_prime, CeFH2_prime, CeFH_prime, CeHF_prime, CeFF_prime, CeFH_hat, CeFH1_hat, CeHF_hat = compute_ce(pe, tb_mat,j0_hat, j0_prime, jmbar_hat, jmbar_prime,jxbar_hat, jxbar_prime, ParaList, df, tax_scenario)
    
    ## compute value of exports and imports
    VgHH, VgFF, VgFH1_prime, VgFH2_prime, VgFH_prime, VgHF_prime, VgFH, VgHF = comp_vg(pe, tb_mat, CeFH1_hat, j0_prime, jxbar_prime, CeFH_hat, CeHF_hat, df, tax_scenario, ParaList)
    
    Ge_prime = CeHH_prime + CeFH_prime;
    Gestar_prime = CeFF_prime + CeHF_prime;
    Ce_prime = CeHH_prime + CeHF_prime;
    Ce_hat = Ce_prime/df['Ce'];
    Cestar_prime = CeFF_prime + CeFH_prime;
    Qeworld_prime=Qe_prime+Qestar_prime;
    pai_g = VgFH - (pe + tb_mat[0]) * df['CeFH'] / (1 - alpha);
    subsidy_ratio = (tb_mat[0]/pe * (1 - alpha)) / (1 + tb_mat[0]/pe * (1 - alpha));
      
    ## compute Ve values
    Ve_prime, Vestar_prime = comp_ve(pe, tb_mat, Ce_prime, tax_scenario, CeHH_prime, CeHF_prime, CeFH_prime, CeFH1_prime, CeFH2_prime, CeFF_prime, Cestar_prime)
    
    Vg, Vg_prime, Vgstar, Vgstar_prime = comp_vgfin(pe, tb_mat, Ce_hat, Cestar_prime, CeHH_prime, CeHF_prime, CeFH_prime, CeFF_prime,VgFH_prime, ParaList, df, tax_scenario)
    
    Lg, Lg_prime, Lgstar, Lgstar_prime = comp_lg(pe, tb_mat, VgFH_prime, Ce_hat, Ge_prime, Gestar_prime, CeHH_prime, CeFH_prime, CeHF_prime, CeFF_prime, ParaList, df, tax_scenario)
    
    leakage1, leakage2, leakage3 = comp_leak(Qestar_prime, Gestar_prime, Cestar_prime, Qeworld_prime, df)
    
    delta_Le, delta_Lestar, delta_U, pe = comp_delta(Lg, Lg_prime, Lgstar, Lgstar_prime, Qeworld_prime, Vg, Vgstar, df, j0_prime, jmbar_prime, jxbar_prime, pe, petbte, tb_mat, tax_scenario, varphi, ParaList)
    
    minuswelfare.welfare = delta_U/Vg;  
    ob=-minuswelfare.welfare;
    return ob


In [199]:
def minuswelfare_purete(te, tb_mat, varphi, tax_scenario, ParaList, df):
     #solve for equilibrium
    
    data = (tb_mat, te, varphi, tax_scenario, ParaList, df)
    pe = fsolve(equilibrium_hatalgebra,1,args=data)
    pe=pe[0]
    # print('pe='+str(pe))
    if pe-te<0 and tax_scenario['tax_sce']=='purete':
        pe=te;
        
    alpha, theta, sigma, sigmastar, epsilonD,epsilonDstar, epsilonS, epsilonSstar, beta, gamma, logit = ParaList
    ## compute extraction tax, and jbar's
    te, jxbar_hat, jmbar_hat, j0_hat, tb_mat = computejbar(ParaList, pe, te, varphi, tb_mat, tax_scenario, df)
    
    jxbar_prime = jxbar_hat * df['jxbar'];
    jmbar_prime = jmbar_hat * df['jmbar'];
    j0_prime = j0_hat * df['jxbar'];
    
    #if te is too large, Home stop producing
    petbte = pe + tb_mat[0] - te;
    z = pe + tb_mat[0] >= te;
    petbte = petbte * z;

    Qe_prime, Qestar_prime = compute_qe(petbte, epsilonS, epsilonSstar, logit, beta, gamma, pe, df)
    
    CeHH_prime, CeFH1_prime, CeFH2_prime, CeFH_prime, CeHF_prime, CeFF_prime, CeFH_hat, CeFH1_hat, CeHF_hat = compute_ce(pe, tb_mat,j0_hat, j0_prime, jmbar_hat, jmbar_prime,jxbar_hat, jxbar_prime, ParaList, df, tax_scenario)
    
    VgHH, VgFF, VgFH1_prime, VgFH2_prime, VgFH_prime, VgHF_prime, VgFH, VgHF = comp_vg(pe, tb_mat, CeFH1_hat, j0_prime, jxbar_prime, CeFH_hat, CeHF_hat, df, tax_scenario, ParaList)
    
    Ge_prime = CeHH_prime + CeFH_prime;
    #Ge_hat = Ge_prime/df['Ge'];
    Gestar_prime = CeFF_prime + CeHF_prime;
    Ce_prime = CeHH_prime + CeHF_prime;
    Ce_hat = Ce_prime/df['Ce'];
    Cestar_prime = CeFF_prime + CeFH_prime;
    Qeworld_prime=Qe_prime+Qestar_prime;
    pai_g = VgFH - (pe + tb_mat[0]) * df['CeFH'] / (1 - alpha);
    subsidy_ratio = (tb_mat[0]/pe * (1 - alpha)) / (1 + tb_mat[0]/pe * (1 - alpha));
    
    ## compute Ve values
    Ve_prime, Vestar_prime = comp_ve(pe, tb_mat, Ce_prime, tax_scenario, CeHH_prime, CeHF_prime, CeFH_prime, CeFH1_prime, CeFH2_prime, CeFF_prime, Cestar_prime)
    
    ## compute more Vg values
    Vg, Vg_prime, Vgstar, Vgstar_prime = comp_vgfin(pe, tb_mat, Ce_hat, Cestar_prime, CeHH_prime, CeHF_prime, CeFH_prime, CeFF_prime,VgFH_prime, ParaList, df, tax_scenario)
        
    Lg, Lg_prime, Lgstar, Lgstar_prime = comp_lg(pe, tb_mat, VgFH_prime, Ce_hat, Ge_prime, Gestar_prime, CeHH_prime, CeFH_prime, CeHF_prime, CeFF_prime, ParaList, df, tax_scenario)
    
    leakage1, leakage2, leakage3 = comp_leak(Qestar_prime, Gestar_prime, Cestar_prime, Qeworld_prime, df)
        
    delta_Le, delta_Lestar, delta_U, pe = comp_delta(Lg, Lg_prime, Lgstar, Lgstar_prime, Qeworld_prime, Vg, Vgstar, df, j0_prime, jmbar_prime, jxbar_prime, pe, petbte, tb_mat, tax_scenario, varphi, ParaList)

    minuswelfare.welfare = delta_U/Vg;  
    ob=-minuswelfare.welfare;
    return ob
          

In [393]:
def minimization(df, tb_mat, te, varphi, tax_scenario, ParaList):
    alpha, theta, sigma, sigmastar, epsilonD,epsilonDstar, epsilonS, epsilonSstar, beta, gamma, logit = ParaList

    if tax_scenario['tax_sce']=='Baseline':
        tb=0
        prop=1
        te=0
        return pd.Series({'tb': tb, 'prop': prop, 'te': te})
    elif tax_scenario['tax_sce']=='PC_hybrid':
        res = minimize(minuswelfare, [0, 1, 1], bounds=[(0,None),(0,1),(0, 1)],method='trust-constr',args=(te, varphi, tax_scenario, ParaList, df));
        tb_mat = res.x;
        tb=tb_mat[0];
        prop=tb_mat[1];   
        prop2=tb_mat[2];
        te=tb
        return pd.Series({'tb': tb, 'prop': prop, 'prop2': prop2,'te': te})
    
    elif tax_scenario['tax_sce']=='EPC_hybrid':
        res = minimize(minuswelfare, [0, 1, 1], bounds=[(0,None),(0,1),(0, 1)],method='trust-constr',args=(te, varphi, tax_scenario, ParaList, df));
        tb_mat = res.x;
        tb=tb_mat[0];
        prop=tb_mat[1];   
        prop2=tb_mat[2];
        te=varphi
        return pd.Series({'tb': tb, 'prop': prop, 'prop2': prop2,'te': te})
  
    elif tax_scenario['tax_sce']=='EP_hybrid':
        res = minimize(minuswelfare, [0, 1], bounds=[(0,None),(0,None)],method='L-BFGS-B', args=(te, varphi, tax_scenario, ParaList, df));
        tb_mat = res.x;
        tb=tb_mat[0];
        te=tb_mat[1];   
        prop=te-tb;
        return pd.Series({'tb': tb, 'prop': prop, 'te': te})
    elif tax_scenario['tax_sce']=='Unilateral' or tax_scenario['tax_sce']=='puretc' or tax_scenario['tax_sce']=='puretp' or tax_scenario['tax_sce']=='EC_hybrid' :
        res = minimize(minuswelfare, [0, 1], method='nelder-mead', args=(te, varphi, tax_scenario, ParaList, df));
        tb_mat = res.x;
        tb=tb_mat[0];
        prop=tb_mat[1];
        #print("tax returned is", tb_mat, "flag is", res.success, "tax scenario is", tax_scenario['tax_sce'])
        if tax_scenario['tax_sce']=='puretc' or tax_scenario['tax_sce']=='puretp':
            te=tb
        elif tax_scenario['tax_sce']=='Unilateral' or tax_scenario['tax_sce']=='EC_hybrid':
            te=varphi
        return pd.Series({'tb': tb, 'prop': prop, 'te': te})
    elif tax_scenario['tax_sce']=='purete':
        res = minimize(minuswelfare_purete, 0, method='nelder-mead', args=(tb_mat, varphi, tax_scenario, ParaList, df));
        te = res.x[0]
        tb_mat = [0,1];
        tb=tb_mat[0];
        prop=tb_mat[1];  
        return pd.Series({'tb': tb, 'prop': prop, 'te': te})
   
    
def callback_pe(pe, df, varphi, tax_scenario, ParaList):
    alpha, theta, sigma, sigmastar, epsilonD,epsilonDstar, epsilonS, epsilonSstar, beta, gamma, logit = ParaList

    ## optimal values
    # jxbar_hat =   (1 - df['jxbar']) ** (-1) / (((1 - df['jxbar']) ** (-1) - 1) + (1 + (1 - alpha) * df['tb']/pe) ** (-theta) * (1 + df['tb']/pe) ** ((1 - alpha) * theta));
    jxbar_hat = pe**(-alpha*theta) * (pe+df['tb'])**(-(1-alpha)*theta) / ( df['jxbar'] * pe**(-alpha*theta) * (pe+df['tb'])**(-(1-alpha)*theta) + (1-df['jxbar']) * (pe + (1-alpha) * df['tb'])**(-theta));
    j0_hat = (pe+df['tb'])**(-(1-alpha)*theta) / (df['jxbar'] * (pe+df['tb'])**(-(1-alpha)*theta) + (1-df['jxbar']) * pe**(-(1-alpha)*theta) );
    jmbar_hat = 1 ;
    
    if tax_scenario['tax_sce']=='Unilateral':
        df['te']=varphi;
        df['prop']=1;
        
    if tax_scenario['tax_sce']=='purete':
        jxbar_hat = 1;   
        jmbar_hat = 1;

    if tax_scenario['tax_sce']=='puretc':
        df['te']=df['tb'];
        jxbar_hat = 1;
        jmbar_hat = 1;
        df['prop']=1;
    
    if tax_scenario['tax_sce']=='puretp':
        df['te']=df['tb'];
        jxbar_hat =  (1 - df['jxbar']) ** (-1) / ((1 - df['jxbar']) ** (-1) - 1 + (1 + df['tb']/pe) ** (theta * (1 - alpha)));
        jmbar_hat = (1 + ((1 - df['jmbar']) ** (-1) - 1) ** (-1)) / (1 + ((1 - df['jmbar']) ** (-1) - 1) ** (-1) * (1 + df['tb']/pe) ** (theta * (1 - alpha)));
        df['prop']=1;

    
    if tax_scenario['tax_sce']=='EC_hybrid':
        df['te']=varphi;
        jxbar_hat = 1;
        jmbar_hat = 1;
        df['prop']=1;

    
    if tax_scenario['tax_sce']=='PC_hybrid':
        df['te']=df['tb'];
        if df['prop']<0:
            df['prop']=0
        jxbar_hat =  (1 - df['jxbar']) ** (-1) / ((1 - df['jxbar']) ** (-1) - 1 + (1 + (df['tb'] - df['prop2'] * df['tb'])/pe) ** (theta * (1 - alpha)));
        jmbar_hat = (1 + ((1 - df['jmbar']) ** (-1) - 1) ** (-1)) / (1 + ((1 - df['jmbar']) ** (-1) - 1) ** (-1) * ((pe + df['tb'])/(pe + df['prop'] * df['tb'])) ** (theta * (1 - alpha)));
    
    if tax_scenario['tax_sce']=='EP_hybrid':
        # df['te']=df['tb']+df['prop'];
        # if df['prop']<0:
        #     df['prop']=0
        jxbar_hat =  (1 - df['jxbar']) ** (-1) / ((1 - df['jxbar']) ** (-1) - 1 + (1 + df['tb']/pe) ** (theta * (1 - alpha)));
        jmbar_hat = (1 + ((1 - df['jmbar']) ** (-1) - 1) ** (-1)) / (1 + ((1 - df['jmbar']) ** (-1) - 1) ** (-1) * (1 + df['tb']/pe) ** (theta * (1 - alpha)));
  
    if tax_scenario['tax_sce']=='EPC_hybrid':
        df['te']=varphi;
        if df['prop']<0:
            df['prop']=0
        jxbar_hat =  (1 - df['jxbar']) ** (-1) / ((1 - df['jxbar']) ** (-1) - 1 + (1 + (df['tb'] - df['prop2'] * df['tb'])/pe) ** (theta * (1 - alpha)));
        jmbar_hat = (1 + ((1 - df['jmbar']) ** (-1) - 1) ** (-1)) / (1 + ((1 - df['jmbar']) ** (-1) - 1) ** (-1) * ((pe + df['tb'])/(pe + df['prop'] * df['tb'])) ** (theta * (1 - alpha)));
              
    
    jxbar_prime = jxbar_hat * df['jxbar'];
    jmbar_prime = jmbar_hat * df['jmbar'];
    j0_prime = j0_hat * df['jxbar'];

    def tempFunction(i, theta, sigmastar):
        return (i ** ((1 + theta) / theta - 1) * (1 - i) ** ((theta - sigmastar) / theta - 1)) 
    
    Bfunvec1_prime = quad(tempFunction,0,j0_prime, args=(theta, sigmastar))[0];
    Bfunvec2_prime = quad(tempFunction,0,jxbar_prime, args=(theta, sigmastar))[0];

    #if te is too large, HoGe stop producing
    petbte = pe + df['tb'] - df['te'];
    z = pe +  df['tb'] >= df['te'];
    petbte = petbte * z;
    
    Qe_hat = (petbte) ** epsilonS;
    Qestar_hat = pe ** epsilonSstar;
          
    if logit==1:
        epsilonS=beta*(1-gamma)/(1-gamma+gamma*petbte**beta);
        epsilonSstar=beta*(1-gamma)/(1-gamma+gamma*pe**beta);
        Qe_hat = (petbte)**beta/(1-gamma+gamma*(petbte)**beta);
        Qestar_hat = pe**beta/(1-gamma+gamma*pe**beta);
 
    Qe_prime = df['Qe'] * Qe_hat;
    Qestar_prime = df['Qestar'] * Qestar_hat;
    
    CeHH_hat = (pe + df['tb']) ** (-epsilonD) * jmbar_hat ** (1 + (1 - sigma)/theta);
    CeHH_prime = df['CeHH'] * CeHH_hat;
       
    
    # CeFH_hat = (1 + (1 - sigmastar)/theta) * pe ** (-(1 - alpha) * sigmastar) * (pe + df['tb']) ** (-alpha) * Bfunvec_prime/(df['jxbar'] ** (1 +1/theta)) * (1 - df['jxbar']) ** (sigmastar/theta);
    CeFH1_hat = (pe +df['tb'])**(-epsilonDstar) * j0_hat**(1 + (1 - sigmastar)/theta);
    CeFH2_hat = (1 + (1 - sigmastar)/theta) * ((1-df['jxbar'])/df['jxbar'])**(sigmastar/theta) * pe**(-epsilonDstar) * (1 + df['tb']/pe)**(-alpha) * (Bfunvec2_prime - Bfunvec1_prime)/df['jxbar']**(1+(1-sigmastar)/theta);
    CeFH1_prime = df['CeFH'] * CeFH1_hat;
    CeFH2_prime = df['CeFH'] * CeFH2_hat;
    CeFH_hat = CeFH1_hat + CeFH2_hat;
    if tax_scenario['Base']==1:
        CeFH_hat = pe ** (-epsilonDstar) * jxbar_hat ** (1 + (1 - sigmastar)/theta);
    
    if tax_scenario['tax_sce']=='puretp' or tax_scenario['tax_sce']=='EP_hybrid':
        CeFH_hat = (pe + df['tb']) ** (-epsilonDstar) * jxbar_hat ** (1 + (1 - sigmastar)/theta);
    
    if tax_scenario['tax_sce']=='PC_hybrid' or tax_scenario['tax_sce']=='EPC_hybrid':
        CeFH_hat = (pe + df['tb'] - df['prop2'] * df['tb']) ** (-epsilonDstar) * jxbar_hat ** (1 + (1 - sigmastar)/theta);
    
    if np.isnan(CeFH_hat)==True:
        CeFH_hat=0
    CeFH_prime =df['CeFH'] * CeFH_hat;
    
    
    CeHF_hat = (pe + df['tb']) ** (-epsilonD);
    if tax_scenario['tax_sce']=='puretp' or tax_scenario['tax_sce']=='EP_hybrid':
        CeHF_hat = (pe) ** (-epsilonD) * ((1 - jmbar_prime)/(1 - df['jmbar'])) ** (1 + (1 - sigma)/theta);
    
    if tax_scenario['tax_sce']=='PC_hybrid' or tax_scenario['tax_sce']=='EPC_hybrid':
        CeHF_hat = (pe + df['prop'] * df['tb']) ** (-epsilonD) * ((1 - jmbar_prime)/(1 - df['jmbar'])) ** (1 + (1 - sigma)/theta);
    
    CeHF_prime = df['CeHF'] * CeHF_hat;
    
    
    CeFF_prime = df['CeFF'] * ((1 - jxbar_prime)/(1-df['jxbar'])) ** (1 + (1 - sigmastar)/theta) * pe ** (-epsilonDstar);
    
    diff = Qe_prime + Qestar_prime - (CeHH_prime + CeFH_prime + CeHF_prime + CeFF_prime);
    #print('diff='+str(diff))
    return diff
    
    
    
def callback(df, varphi, tax_scenario, ParaList):
    #solve for equilibrium
    if df['prop']<0:
        df['prop']=0
        
    pe = fsolve(callback_pe,1,args=(df, varphi, tax_scenario, ParaList))
    pe=pe[0]
    # print('pe='+str(pe))
    alpha, theta, sigma, sigmastar, epsilonD,epsilonDstar, epsilonS, epsilonSstar, beta, gamma, logit = ParaList
    
   ## optimal values
    # jxbar_hat =   (1 - df['jxbar']) ** (-1) / (((1 - df['jxbar']) ** (-1) - 1) + (1 + (1 - alpha) * df['tb']/pe) ** (-theta) * (1 + df['tb']/pe) ** ((1 - alpha) * theta));
    jxbar_hat = pe**(-alpha*theta) * (pe+df['tb'])**(-(1-alpha)*theta) / ( df['jxbar'] * pe**(-alpha*theta) * (pe+df['tb'])**(-(1-alpha)*theta) + (1-df['jxbar']) * (pe + (1-alpha) * df['tb'])**(-theta));
    j0_hat = (pe+df['tb'])**(-(1-alpha)*theta) / (df['jxbar'] * (pe+df['tb'])**(-(1-alpha)*theta) + (1-df['jxbar']) * pe**(-(1-alpha)*theta) );
    jmbar_hat = 1 ;
    
    if tax_scenario['tax_sce']=='Unilateral':
        df['te']=varphi;
        df['prop']=1;
        
    if tax_scenario['tax_sce']=='purete':
        jxbar_hat = 1;   
        jmbar_hat = 1;

    if tax_scenario['tax_sce']=='puretc':
        df['te']=df['tb'];
        jxbar_hat = 1;
        jmbar_hat = 1;
        df['prop']=1;
    
    if tax_scenario['tax_sce']=='puretp':
        df['te']=df['tb'];
        jxbar_hat =  (1 - df['jxbar']) ** (-1) / ((1 - df['jxbar']) ** (-1) - 1 + (1 + df['tb']/pe) ** (theta * (1 - alpha)));
        jmbar_hat = (1 + ((1 - df['jmbar']) ** (-1) - 1) ** (-1)) / (1 + ((1 - df['jmbar']) ** (-1) - 1) ** (-1) * (1 + df['tb']/pe) ** (theta * (1 - alpha)));
        df['prop']=1;

    
    if tax_scenario['tax_sce']=='EC_hybrid':
        df['te']=varphi;
        jxbar_hat = 1;
        jmbar_hat = 1;
        df['prop']=1;

    
    if tax_scenario['tax_sce']=='PC_hybrid':
        df['te']=df['tb'];
        if df['prop']<0:
            df['prop']=0
        jxbar_hat =  (1 - df['jxbar']) ** (-1) / ((1 - df['jxbar']) ** (-1) - 1 + (1 + (df['tb'] - df['prop2'] * df['tb'])/pe) ** (theta * (1 - alpha)));
        jmbar_hat = (1 + ((1 - df['jmbar']) ** (-1) - 1) ** (-1)) / (1 + ((1 - df['jmbar']) ** (-1) - 1) ** (-1) * ((pe + df['tb'])/(pe + df['prop'] * df['tb'])) ** (theta * (1 - alpha)));
    
    if tax_scenario['tax_sce']=='EP_hybrid':
        # df['te']=df['tb']+df['prop'];
        # if df['prop']<0:
        #     df['prop']=0
        jxbar_hat =  (1 - df['jxbar']) ** (-1) / ((1 - df['jxbar']) ** (-1) - 1 + (1 + df['tb']/pe) ** (theta * (1 - alpha)));
        jmbar_hat = (1 + ((1 - df['jmbar']) ** (-1) - 1) ** (-1)) / (1 + ((1 - df['jmbar']) ** (-1) - 1) ** (-1) * (1 + df['tb']/pe) ** (theta * (1 - alpha)));
              
    if tax_scenario['tax_sce']=='EPC_hybrid':
       df['te']=varphi;
       if df['prop']<0:
            df['prop']=0
       jxbar_hat =  (1 - df['jxbar']) ** (-1) / ((1 - df['jxbar']) ** (-1) - 1 + (1 + (df['tb'] - df['prop2'] * df['tb'])/pe) ** (theta * (1 - alpha)));
       jmbar_hat = (1 + ((1 - df['jmbar']) ** (-1) - 1) ** (-1)) / (1 + ((1 - df['jmbar']) ** (-1) - 1) ** (-1) * ((pe + df['tb'])/(pe + df['prop'] * df['tb'])) ** (theta * (1 - alpha)));
        
    
    ## compute extraction tax, and jbar's
    #te, jxbar_hat, jmbar_hat, j0_hat, tb_mat = computejbar(ParaList, pe, te, varphi, tb_mat, tax_scenario)
    
    jxbar_prime = jxbar_hat * df['jxbar'];
    jmbar_prime = jmbar_hat * df['jmbar'];
    j0_prime = j0_hat * df['jxbar'];

    def tempFunction(i, theta, sigmastar):
        return (i ** ((1 + theta) / theta - 1) * (1 - i) ** ((theta - sigmastar) / theta - 1)) 
    
    Bfunvec1_prime = quad(tempFunction,0,j0_prime, args=(theta, sigmastar))[0];
    Bfunvec2_prime = quad(tempFunction,0,jxbar_prime, args=(theta, sigmastar))[0];

    #if te is too large, HoGe stop producing
    petbte = pe + df['tb'] - df['te'];
    z = pe +  df['tb'] >= df['te'];

    petbte = petbte * z;

    Qe_hat = (petbte) ** epsilonS;
    Qestar_hat = pe ** epsilonSstar;
          
    if logit==1:
        epsilonS=beta*(1-gamma)/(1-gamma+gamma*petbte**beta);
        epsilonSstar=beta*(1-gamma)/(1-gamma+gamma*pe**beta);
        Qe_hat = (petbte)**beta/(1-gamma+gamma*(petbte)**beta);
        Qestar_hat = pe**beta/(1-gamma+gamma*pe**beta);
 
    Qe_prime = df['Qe'] * Qe_hat;
    Qestar_prime = df['Qestar'] * Qestar_hat;
    
    
    CeHH_hat = (pe + df['tb']) ** (-epsilonD) * jmbar_hat ** (1 + (1 - sigma)/theta);
    CeHH_prime = df['CeHH'] * CeHH_hat;
       
    
    # CeFH_hat = (1 + (1 - sigmastar)/theta) * pe ** (-(1 - alpha) * sigmastar) * (pe + df['tb']) ** (-alpha) * Bfunvec_prime/(df['jxbar'] ** (1 +1/theta)) * (1 - df['jxbar']) ** (sigmastar/theta);
    CeFH1_hat = (pe +df['tb'])**(-epsilonDstar) * j0_hat**(1 + (1 - sigmastar)/theta);
    CeFH2_hat = (1 + (1 - sigmastar)/theta) * ((1-df['jxbar'])/df['jxbar'])**(sigmastar/theta) * pe**(-epsilonDstar) * (1 + df['tb']/pe)**(-alpha) * (Bfunvec2_prime - Bfunvec1_prime)/df['jxbar']**(1+(1-sigmastar)/theta);
    CeFH1_prime = df['CeFH'] * CeFH1_hat;
    CeFH2_prime = df['CeFH'] * CeFH2_hat;
    CeFH_hat = CeFH1_hat + CeFH2_hat;
    if tax_scenario['Base']==1:
        CeFH_hat = pe ** (-epsilonDstar) * jxbar_hat ** (1 + (1 - sigmastar)/theta);
    
    if tax_scenario['tax_sce']=='puretp' or tax_scenario['tax_sce']=='EP_hybrid':
        CeFH_hat = (pe + df['tb']) ** (-epsilonDstar) * jxbar_hat ** (1 + (1 - sigmastar)/theta);
    
    if tax_scenario['tax_sce']=='PC_hybrid' or tax_scenario['tax_sce']=='EPC_hybrid':
        CeFH_hat = (pe + df['tb'] - df['prop2'] * df['tb']) ** (-epsilonDstar) * jxbar_hat ** (1 + (1 - sigmastar)/theta);
    
    if np.isnan(CeFH_hat)==True:
        CeFH_hat=0
    CeFH_prime =df['CeFH'] * CeFH_hat;
    
    
    CeHF_hat = (pe + df['tb']) ** (-epsilonD);
    if tax_scenario['tax_sce']=='puretp' or tax_scenario['tax_sce']=='EP_hybrid':
        CeHF_hat = (pe) ** (-epsilonD) * ((1 - jmbar_prime)/(1 - df['jmbar'])) ** (1 + (1 - sigma)/theta);
    
    if tax_scenario['tax_sce']=='PC_hybrid' or tax_scenario['tax_sce']=='EPC_hybrid':
        CeHF_hat = (pe + df['prop'] * df['tb']) ** (-epsilonD) * ((1 - jmbar_prime)/(1 - df['jmbar'])) ** (1 + (1 - sigma)/theta);
    
    CeHF_prime = df['CeHF'] * CeHF_hat;
    
    
    CeFF_prime = df['CeFF'] * ((1 - jxbar_prime)/(1-df['jxbar'])) ** (1 + (1 - sigmastar)/theta) * pe ** (-epsilonDstar);
    
    ##
    VgHH = df['CeHH']/(1 - alpha);
    VgFF = df['CeFF']/(1 - alpha);
    
    VgFH = df['CeFH'] /(1 - alpha);
    # VgFH_prime = VgFH * pe ** ((1 - sigmastar) * (1 - alpha)) * (1 - (1 - jxbar_prime) ** (1 + (1 - sigmastar)/theta))/ (df['jxbar'] * (1 - df['jxbar']) ** ( (1-sigmastar)/theta));
    VgFH1_hat = (pe + df['tb']) * CeFH1_hat;
    VgFH2_hat = pe**(1 - epsilonDstar) * ((1-j0_prime)**(1+(1-sigmastar)/theta) - (1-jxbar_prime)**(1+(1-sigmastar)/theta))/ (df['jxbar']  * (1 - df['jxbar'] )**( (1-sigmastar)/theta));
    VgFH1_prime = VgFH * VgFH1_hat;
    VgFH2_prime = VgFH * VgFH2_hat;
    VgFH_hat = VgFH1_hat + VgFH2_hat;
    if tax_scenario['Base']==1:
        VgFH_hat = pe * CeFH_hat;
    if tax_scenario['tax_sce']=='puretp' or tax_scenario['tax_sce']=='EP_hybrid':
        VgFH_hat = (pe + df['tb']) * CeFH_hat;
    if tax_scenario['tax_sce']=='PC_hybrid' or tax_scenario['tax_sce']=='EPC_hybrid':
        VgFH_hat = (pe + df['tb'] - df['prop2'] * df['tb']) * CeFH_hat;
    if np.isnan(VgFH_hat)==True:
            VgFH_hat=0
    VgFH_prime = VgFH * VgFH_hat;  

    VgHF = df['CeHF']/(1 - alpha);
    VgHF_hat = (pe + alpha * df['tb']) * CeHF_hat;
    if tax_scenario['tax_sce']=='puretp' or tax_scenario['tax_sce']=='EP_hybrid':
        VgHF_hat = pe * CeHF_hat;
    if tax_scenario['tax_sce']=='PC_hybrid' or tax_scenario['tax_sce']=='EPC_hybrid':
        VgHF_hat = (pe + alpha * df['prop'] * df['tb']) * CeHF_hat; 
    VgHF_prime = VgHF * VgHF_hat;
    Ge_prime = CeHH_prime + CeFH_prime;
    Ge_hat = Ge_prime/df['Ge'];
    Gestar_prime = CeFF_prime + CeHF_prime;
    Ce_prime = CeHH_prime + CeHF_prime;
    Ce_hat = Ce_prime/df['Ce'];
    Cestar_prime = CeFF_prime + CeFH_prime;
    Qeworld_prime=Qe_prime+Qestar_prime;
    pai_g = VgFH - (pe + df['tb']) * df['CeFH'] / (1 - alpha);
    subsidy_ratio = (df['tb']/pe * (1 - alpha)) / (1 + df['tb']/pe * (1 - alpha));
    
    Ve_prime=(pe+df['tb']) * Ce_prime;
    if tax_scenario['tax_sce']=='puretp' or tax_scenario['tax_sce']=='EP_hybrid':
        Ve_prime = (pe+df['tb']) * CeHH_prime + pe * CeHF_prime;
    if tax_scenario['tax_sce']=='PC_hybrid' or tax_scenario['tax_sce']=='EPC_hybrid':
        Ve_prime = (pe+df['tb']) * CeHH_prime + (pe + df['prop']*df['tb']) * CeHF_prime;
    
    Vestar_prime= (pe+df['tb']) * CeFH_prime  + pe * CeFF_prime;
    Vestar_prime= (pe+df['tb']) * CeFH1_prime + pe * CeFH2_prime + pe * CeFF_prime;
    if tax_scenario['tax_sce']=='puretc' or tax_scenario['tax_sce']=='EC_hybrid' :
        Vestar_prime = pe * Cestar_prime;
    if tax_scenario['tax_sce']=='PC_hybrid':
        Vestar_prime = (pe + df['tb'] - df['prop2']*df['tb']) * CeFH_prime + pe * CeFF_prime;    
    
    Vg = df['Ce'] /(1-alpha);
    Vg_prime_hat = (pe + df['tb']) * Ce_hat;
    Vg_prime = Vg * Vg_prime_hat;
    if tax_scenario['tax_sce']=='puretp' or tax_scenario['tax_sce']=='EP_hybrid':
        Vg_prime = CeHH_prime/(1-alpha) * (pe + df['tb']) + CeHF_prime/(1-alpha) * pe; 
    if tax_scenario['tax_sce']=='PC_hybrid' or tax_scenario['tax_sce']=='EPC_hybrid':
        Vg_prime = CeHH_prime/(1-alpha) * (pe + df['tb']) + CeHF_prime/(1-alpha) * (pe + df['prop'] * df['tb']);   
    
    Vgstar = df['Cestar'] /(1-alpha);
    Vgstar_prime = VgFH_prime + CeFF_prime/(1-alpha)* pe;
    if tax_scenario['tax_sce']=='puretc' or tax_scenario['tax_sce']=='EC_hybrid' :
        Vgstar_prime = Cestar_prime/(1-alpha)* pe;
    if tax_scenario['tax_sce']=='PC_hybrid':
        Vgstar_prime = CeFF_prime/(1-alpha)* pe + CeFH_prime/(1-alpha)* (pe + df['tb'] - df['prop2']*df['tb']);
    
    Lg = alpha/(1-alpha) * df['Ge'];
    Lg_prime = alpha/(1-alpha) * (pe + df['tb']) * Ge_prime;
    if tax_scenario['tax_sce']=='puretc' or tax_scenario['tax_sce']=='EC_hybrid':
        Lg_prime = alpha/(1-alpha) * (pe + df['tb']) * CeHH_prime + alpha/(1-alpha) * pe * CeFH_prime;  
    if tax_scenario['tax_sce']=='PC_hybrid' or tax_scenario['tax_sce']=='EPC_hybrid':
        Lg_prime = alpha/(1-alpha) * (pe + df['tb']) * CeHH_prime + alpha/(1-alpha) * (pe + df['tb'] - df['prop2'] * df['tb']) * CeFH_prime;    
    
    Lgstar = alpha/(1-alpha) * df['Gestar'];
    Lgstar_prime = alpha/(1-alpha) * (pe+df['tb']) * CeHF_prime +alpha/(1-alpha) * pe * CeFF_prime;
    if tax_scenario['tax_sce']=='puretp' or tax_scenario['tax_sce']=='EP_hybrid':
        Lgstar_prime = alpha/(1-alpha) * pe * Gestar_prime;
    if tax_scenario['tax_sce']=='PC_hybrid' or tax_scenario['tax_sce']=='EPC_hybrid':
        Lgstar_prime = alpha/(1-alpha) * (pe + df['prop']*df['tb']) * CeHF_prime + alpha/(1-alpha) * pe * CeFF_prime;

    delta_Le = (epsilonS/(epsilonS + 1)) * df['Qe'] * (petbte**(epsilonS + 1) - 1);
    delta_Lestar = (epsilonSstar/(epsilonSstar + 1)) * df['Qestar'] * (pe**(epsilonSstar + 1) - 1);
    
    def Func(a, beta, gamma):
        return (((1-gamma)*a**beta)/(1-gamma+gamma*a**beta)**2)
    if logit==1:
        delta_Le = beta * df['Qe'] * quad(Func,1,petbte, args=(beta, gamma))[0];
        delta_Lestar = beta * df['Qestar'] * quad(Func,1,pe, args=(beta, gamma))[0];
    
    leakage1 = -(Qestar_prime - df['Qestar'])/(Qeworld_prime - df['Qeworld']);
    leakage2 = -(Gestar_prime - df['Gestar'])/(Qeworld_prime - df['Qeworld']);
    leakage3 = -(Cestar_prime - df['Cestar'])/(Qeworld_prime - df['Qeworld']);   
    chg_extraction=Qestar_prime-df['Qestar'];
    chg_production=Gestar_prime-df['Gestar'];
    chg_consumption=Cestar_prime-df['Cestar'];
    chg_Qeworld=Qeworld_prime-df['Qeworld'];

    # delta_U = -delta_Le - delta_Lestar - (Lg_prime - Lg) - (Lgstar_prime - Lgstar) \
    #        + sigma/(sigma-1)* (Vg_prime-Vg) + sigmastar/(sigmastar-1)* (Vgstar_prime-Vgstar) \
    #        - varphi * (Qeworld_prime - df['Qeworld']);
       
    delta_U = -delta_Le - delta_Lestar - (Lg_prime - Lg) - (Lgstar_prime - Lgstar) \
           + Vg * (alpha - 1) * math.log(pe + df['tb']) + Vgstar * (1/theta) * math.log(df['jxbar']/j0_prime * (pe+df['tb'])**(-(1-alpha)*theta)) \
           - varphi * (Qeworld_prime - df['Qeworld']);
    # print("pe")
    # print(pe)
    # print("tb")
    # print(df['tb'])
    if pe<0:
        pe=0.0001
    if tax_scenario['tax_sce']=='puretc' or tax_scenario['tax_sce']=='purete' or tax_scenario['tax_sce']=='EC_hybrid':
        delta_U = -delta_Le - delta_Lestar - (Lg_prime - Lg) - (Lgstar_prime - Lgstar) \
           + Vg * (alpha - 1) * math.log(pe +  df['tb']) + Vgstar * (alpha - 1) * math.log(pe) \
           - varphi * (Qeworld_prime - df['Qeworld']);

    if tax_scenario['tax_sce']=='puretp' or tax_scenario['tax_sce']=='EP_hybrid':
        delta_U = -delta_Le - delta_Lestar - (Lg_prime - Lg) - (Lgstar_prime - Lgstar) \
               +  Vg * ((alpha - 1) * math.log(pe + df['tb']) + 1/theta * math.log(df['jmbar']/jmbar_prime)) \
               +  Vgstar * ((alpha - 1) * math.log(pe + df['tb']) + 1/theta * math.log(df['jxbar']/jxbar_prime)) \
               - varphi * (Qeworld_prime - df['Qeworld']);

    if  tax_scenario['tax_sce']=='PC_hybrid' or tax_scenario['tax_sce']=='EPC_hybrid':
        delta_U = -delta_Le - delta_Lestar - (Lg_prime - Lg) - (Lgstar_prime - Lgstar) \
               +  Vg * ((alpha - 1) * math.log(pe + df['tb']) + 1/theta * math.log(df['jmbar']/jmbar_prime)) \
               +  Vgstar * ((alpha - 1) * math.log(pe + df['tb'] - df['prop2']*df['tb']) + 1/theta * math.log(df['jxbar']/jxbar_prime)) \
               -  varphi * (Qeworld_prime - df['Qeworld']);
    
    
    welfare = delta_U/Vg*100;  
    welfare_noexternality = (delta_U + varphi * (Qeworld_prime - df['Qeworld']) )/Vg*100;  
    
    return(pd.Series({ 'varphi': varphi, 'pe': pe, 'jxbar_prime': jxbar_prime, 'jmbar_prime': jmbar_prime, 'j0_prime': j0_prime, \
                      'Qe_prime': Qe_prime, 'Qestar_prime': Qestar_prime, 'Qeworld_prime': Qeworld_prime, \
                      'CeHH_prime': CeHH_prime,'CeFH_prime': CeFH_prime, 'CeHF_prime': CeHF_prime, 'CeFF_prime': CeFF_prime,\
                      'Ge_prime': Ge_prime,'Ce_prime': Ce_prime, 'Gestar_prime': Gestar_prime,'Cestar_prime': Cestar_prime, \
                      'VgFH_prime': VgFH_prime, 'VgHF_prime': VgHF_prime, \
                      'VgFH1_prime': VgFH1_prime, 'VgFH2_prime': VgFH2_prime, \
                      'CeFH1_prime': CeFH1_prime, 'CeFH2_prime': CeFH2_prime, \
                      'Vg_prime': Vg_prime, 'Vgstar_prime': Vgstar_prime, \
                      'Lg_prime': Lg_prime, 'Lgstar_prime': Lgstar_prime, \
                      'Ve_prime': Ve_prime, 'Vestar_prime': Vestar_prime, \
                      'delta_Le': delta_Le, 'delta_Lestar': delta_Lestar, \
                      'leakage1': leakage1, 'leakage2': leakage2,'leakage3': leakage3, \
                      'leakage3': leakage3, 'chg_extraction': chg_extraction, 'chg_production': chg_production, \
                      'chg_consumption': chg_consumption,'chg_Qeworld':chg_Qeworld, 'pai_g': pai_g, \
                      'subsidy_ratio': subsidy_ratio, 'welfare':welfare, 'welfare_noexternality':welfare_noexternality}))


In [543]:
class taxrow:
    def __init__(self, df, tax_scenario, varphi, ParaList):
        self.df = df
        self.tax_scenario = tax_scenario
        self.varphi = varphi
        self.ParaList = ParaList
        self.results = 0
        
    def opt_tax(self):
        alpha, theta, sigma, sigmastar, epsilonD,epsilonDstar, epsilonS, epsilonSstar, beta, gamma, logit = self.ParaList
        tax_scenario = self.tax_scenario
        varphi = self.varphi
        df = self.df
        te = varphi

        if tax_scenario['tax_sce']=='Baseline':
            self.tb=0
            self.prop=1
            self.te=0
        elif tax_scenario['tax_sce']=='PC_hybrid':
            res = minimize(self.minuswelfare, [0, 1, 1], bounds=[(0,None),(0,1),(0, 1)],method='trust-constr',args=(te, varphi, tax_scenario, ParaList, df));
            tb_mat = res.x;
            self.tb=tb_mat[0];
            self.prop=tb_mat[1];   
            self.prop2=tb_mat[2];
            self.te=self.tb
    
        elif tax_scenario['tax_sce']=='EPC_hybrid':
            res = minimize(self.minuswelfare, [0, 1, 1], bounds=[(0,None),(0,1),(0, 1)],method='trust-constr',args=(te, varphi, tax_scenario, ParaList, df));
            tb_mat = res.x;
            self.tb=tb_mat[0];
            self.prop=tb_mat[1];   
            self.prop2=tb_mat[2];
            self.te=varphi
  
        elif tax_scenario['tax_sce']=='EP_hybrid':
            res = minimize(self.minuswelfare, [0, 1], bounds=[(0,None),(0,None)],method='L-BFGS-B', args=(te, varphi, tax_scenario, ParaList, df));
            tb_mat = res.x;
            self.tb=tb_mat[0];
            self.te=tb_mat[1];   
            self.prop=te-self.tb;
        elif tax_scenario['tax_sce']=='Unilateral' or tax_scenario['tax_sce']=='puretc' or tax_scenario['tax_sce']=='puretp' or tax_scenario['tax_sce']=='EC_hybrid' :
            res = minimize(self.minuswelfare, [0, 1], method='nelder-mead', args=(te, varphi, tax_scenario, ParaList, df));
            tb_mat = res.x;
            self.tb=tb_mat[0];
            self.prop=tb_mat[1];
            #print("tax returned is", tb_mat, "flag is", res.success, "tax scenario is", tax_scenario['tax_sce'])
            if tax_scenario['tax_sce']=='puretc' or tax_scenario['tax_sce']=='puretp':
                self.te=self.tb
            elif tax_scenario['tax_sce']=='Unilateral' or tax_scenario['tax_sce']=='EC_hybrid':
                self.te=self.varphi
        elif tax_scenario['tax_sce']=='purete':
            tb_mat = [0,1];
            res = minimize(self.minuswelfare_purete, 0, method='nelder-mead', args=(tb_mat, varphi, tax_scenario, ParaList, df));
            self.te = res.x[0]
            tb_mat = [0,1];
            self.tb=tb_mat[0];
            self.prop=tb_mat[1];  
            
    def minuswelfare(self, tb_mat, te, varphi, tax_scenario, ParaList, df):
         #solve for equilibrium
        data = (tb_mat, te, varphi, tax_scenario, ParaList, df)
        pe = fsolve(self.equilibrium_hatalgebra,1,args=data, full_output = True)
        pe=pe[0][0]
        if pe-te<0 and tax_scenario['tax_sce']=='purete':
            pe=te;
    
 
        alpha, theta, sigma, sigmastar, epsilonD,epsilonDstar, epsilonS, epsilonSstar, beta, gamma, logit = ParaList
    
        ## compute extraction tax, and jbar_hat's
        te, jxbar_hat, jmbar_hat, j0_hat, tb_mat = computejbar(ParaList, pe, te, varphi, tb_mat, tax_scenario, df)
    
        jxbar_prime = jxbar_hat * df['jxbar'];
        jmbar_prime = jmbar_hat * df['jmbar'];
        j0_prime = j0_hat * df['jxbar'];

        #if te is too large, Home stop producing
        petbte = pe + tb_mat[0] - te;
        z = pe + tb_mat[0] >= te;
        petbte = petbte * z;
    
        ## home and foreign extractions
        Qe_prime, Qestar_prime = compute_qe(petbte, epsilonS, epsilonSstar, logit, beta, gamma, pe, df)
    
        ## compute consumptions around the world
        CeHH_prime, CeFH1_prime, CeFH2_prime, CeFH_prime, CeHF_prime, CeFF_prime, CeFH_hat, CeFH1_hat, CeHF_hat = compute_ce(pe, tb_mat,j0_hat, j0_prime, jmbar_hat, jmbar_prime,jxbar_hat, jxbar_prime, ParaList, df, tax_scenario)
    
        ## compute value of exports and imports
        VgHH, VgFF, VgFH1_prime, VgFH2_prime, VgFH_prime, VgHF_prime, VgFH, VgHF = comp_vg(pe, tb_mat, CeFH1_hat, j0_prime, jxbar_prime, CeFH_hat, CeHF_hat, df, tax_scenario, ParaList)
    
        Ge_prime = CeHH_prime + CeFH_prime;
        Gestar_prime = CeFF_prime + CeHF_prime;
        Ce_prime = CeHH_prime + CeHF_prime;
        Ce_hat = Ce_prime/df['Ce'];
        Cestar_prime = CeFF_prime + CeFH_prime;
        Qeworld_prime=Qe_prime+Qestar_prime;
        pai_g = VgFH - (pe + tb_mat[0]) * df['CeFH'] / (1 - alpha);
        subsidy_ratio = (tb_mat[0]/pe * (1 - alpha)) / (1 + tb_mat[0]/pe * (1 - alpha));
      
        ## compute Ve values
        Ve_prime, Vestar_prime = comp_ve(pe, tb_mat, Ce_prime, tax_scenario, CeHH_prime, CeHF_prime, CeFH_prime, CeFH1_prime, CeFH2_prime, CeFF_prime, Cestar_prime)
    
        Vg, Vg_prime, Vgstar, Vgstar_prime = comp_vgfin(pe, tb_mat, Ce_hat, Cestar_prime, CeHH_prime, CeHF_prime, CeFH_prime, CeFF_prime,VgFH_prime, ParaList, df, tax_scenario)
    
        Lg, Lg_prime, Lgstar, Lgstar_prime = comp_lg(pe, tb_mat, VgFH_prime, Ce_hat, Ge_prime, Gestar_prime, CeHH_prime, CeFH_prime, CeHF_prime, CeFF_prime, ParaList, df, tax_scenario)
    
        leakage1, leakage2, leakage3 = comp_leak(Qestar_prime, Gestar_prime, Cestar_prime, Qeworld_prime, df)
    
        delta_Le, delta_Lestar, delta_U, pe = comp_delta(Lg, Lg_prime, Lgstar, Lgstar_prime, Qeworld_prime, Vg, Vgstar, df, j0_prime, jmbar_prime, jxbar_prime, pe, petbte, tb_mat, tax_scenario, varphi, ParaList)
        
        chg_extraction=Qestar_prime-df['Qestar']
        chg_production=Gestar_prime-df['Gestar']
        chg_consumption=Cestar_prime-df['Cestar']
        chg_Qeworld=Qeworld_prime-df['Qeworld']
        
        welfare = delta_U/Vg*100
        welfare_noexternality = (delta_U + varphi * (Qeworld_prime - df['Qeworld']) )/Vg*100
    
        self.results = (pd.Series({ 'varphi': varphi, 'pe': pe, 'jxbar_prime': jxbar_prime, 'jmbar_prime': jmbar_prime, 'j0_prime': j0_prime, \
                      'Qe_prime': Qe_prime, 'Qestar_prime': Qestar_prime, 'Qeworld_prime': Qeworld_prime, \
                      'CeHH_prime': CeHH_prime,'CeFH_prime': CeFH_prime, 'CeHF_prime': CeHF_prime, 'CeFF_prime': CeFF_prime,\
                      'Ge_prime': Ge_prime,'Ce_prime': Ce_prime, 'Gestar_prime': Gestar_prime,'Cestar_prime': Cestar_prime, \
                      'VgFH_prime': VgFH_prime, 'VgHF_prime': VgHF_prime, \
                      'VgFH1_prime': VgFH1_prime, 'VgFH2_prime': VgFH2_prime, \
                      'CeFH1_prime': CeFH1_prime, 'CeFH2_prime': CeFH2_prime, \
                      'Vg_prime': Vg_prime, 'Vgstar_prime': Vgstar_prime, \
                      'Lg_prime': Lg_prime, 'Lgstar_prime': Lgstar_prime, \
                      'Ve_prime': Ve_prime, 'Vestar_prime': Vestar_prime, \
                      'delta_Le': delta_Le, 'delta_Lestar': delta_Lestar, \
                      'leakage1': leakage1, 'leakage2': leakage2,'leakage3': leakage3, \
                      'leakage3': leakage3, 'chg_extraction': chg_extraction, 'chg_production': chg_production, \
                      'chg_consumption': chg_consumption,'chg_Qeworld':chg_Qeworld, 'pai_g': pai_g, \
                      'subsidy_ratio': subsidy_ratio, 'welfare':welfare, 'welfare_noexternality':welfare_noexternality}))
        minuswelfare.welfare = delta_U/Vg
        ob=-minuswelfare.welfare
        return ob

    
    def minuswelfare_purete(self, te, tb_mat, varphi, tax_scenario, ParaList, df):
         #solve for equilibrium

        data = (tb_mat, te, varphi, tax_scenario, ParaList, df)
        pe = fsolve(self.equilibrium_hatalgebra,1,args=data)
        pe=pe[0]
        # print('pe='+str(pe))
        if pe-te<0 and tax_scenario['tax_sce']=='purete':
            pe=te;

        alpha, theta, sigma, sigmastar, epsilonD,epsilonDstar, epsilonS, epsilonSstar, beta, gamma, logit = ParaList
        ## compute extraction tax, and jbar's
        te, jxbar_hat, jmbar_hat, j0_hat, tb_mat = computejbar(ParaList, pe, te, varphi, tb_mat, tax_scenario, df)

        jxbar_prime = jxbar_hat * df['jxbar'];
        jmbar_prime = jmbar_hat * df['jmbar'];
        j0_prime = j0_hat * df['jxbar'];

        #if te is too large, Home stop producing
        petbte = pe + tb_mat[0] - te;
        z = pe + tb_mat[0] >= te;
        petbte = petbte * z;

        Qe_prime, Qestar_prime = compute_qe(petbte, epsilonS, epsilonSstar, logit, beta, gamma, pe, df)

        CeHH_prime, CeFH1_prime, CeFH2_prime, CeFH_prime, CeHF_prime, CeFF_prime, CeFH_hat, CeFH1_hat, CeHF_hat = compute_ce(pe, tb_mat,j0_hat, j0_prime, jmbar_hat, jmbar_prime,jxbar_hat, jxbar_prime, ParaList, df, tax_scenario)
        self.CeHH_prime, self.CeFH_prime, self.CeHF_prime, self.CeFF_prime = CeHH_prime, CeFH_prime, CeHF_prime, CeFF_prime
        
        VgHH, VgFF, VgFH1_prime, VgFH2_prime, VgFH_prime, VgHF_prime, VgFH, VgHF = comp_vg(pe, tb_mat, CeFH1_hat, j0_prime, jxbar_prime, CeFH_hat, CeHF_hat, df, tax_scenario, ParaList)

        Ge_prime = CeHH_prime + CeFH_prime;
        #Ge_hat = Ge_prime/df['Ge'];
        Gestar_prime = CeFF_prime + CeHF_prime;
        Ce_prime = CeHH_prime + CeHF_prime;
        Ce_hat = Ce_prime/df['Ce'];
        Cestar_prime = CeFF_prime + CeFH_prime;
        Qeworld_prime=Qe_prime+Qestar_prime;
        pai_g = VgFH - (pe + tb_mat[0]) * df['CeFH'] / (1 - alpha);
        subsidy_ratio = (tb_mat[0]/pe * (1 - alpha)) / (1 + tb_mat[0]/pe * (1 - alpha));

        ## compute Ve values
        Ve_prime, Vestar_prime = comp_ve(pe, tb_mat, Ce_prime, tax_scenario, CeHH_prime, CeHF_prime, CeFH_prime, CeFH1_prime, CeFH2_prime, CeFF_prime, Cestar_prime)

        ## compute more Vg values
        Vg, Vg_prime, Vgstar, Vgstar_prime = comp_vgfin(pe, tb_mat, Ce_hat, Cestar_prime, CeHH_prime, CeHF_prime, CeFH_prime, CeFF_prime,VgFH_prime, ParaList, df, tax_scenario)

        Lg, Lg_prime, Lgstar, Lgstar_prime = comp_lg(pe, tb_mat, VgFH_prime, Ce_hat, Ge_prime, Gestar_prime, CeHH_prime, CeFH_prime, CeHF_prime, CeFF_prime, ParaList, df, tax_scenario)

        leakage1, leakage2, leakage3 = comp_leak(Qestar_prime, Gestar_prime, Cestar_prime, Qeworld_prime, df)

        delta_Le, delta_Lestar, delta_U, pe = comp_delta(Lg, Lg_prime, Lgstar, Lgstar_prime, Qeworld_prime, Vg, Vgstar, df, j0_prime, jmbar_prime, jxbar_prime, pe, petbte, tb_mat, tax_scenario, varphi, ParaList)
        
        chg_extraction=Qestar_prime-df['Qestar']
        chg_production=Gestar_prime-df['Gestar']
        chg_consumption=Cestar_prime-df['Cestar']
        chg_Qeworld=Qeworld_prime-df['Qeworld']
        
        welfare = delta_U/Vg*100
        welfare_noexternality = (delta_U + varphi * (Qeworld_prime - df['Qeworld']) )/Vg*100
        
        self.results = (pd.Series({'varphi': varphi, 'pe': pe, 'jxbar_prime': jxbar_prime, 'jmbar_prime': jmbar_prime, 'j0_prime': j0_prime, \
              'Qe_prime': Qe_prime, 'Qestar_prime': Qestar_prime, 'Qeworld_prime': Qeworld_prime, \
              'CeHH_prime': CeHH_prime,'CeFH_prime': CeFH_prime, 'CeHF_prime': CeHF_prime, 'CeFF_prime': CeFF_prime,\
              'Ge_prime': Ge_prime,'Ce_prime': Ce_prime, 'Gestar_prime': Gestar_prime,'Cestar_prime': Cestar_prime, \
              'VgFH_prime': VgFH_prime, 'VgHF_prime': VgHF_prime, \
              'VgFH1_prime': VgFH1_prime, 'VgFH2_prime': VgFH2_prime, \
              'CeFH1_prime': CeFH1_prime, 'CeFH2_prime': CeFH2_prime, \
              'Vg_prime': Vg_prime, 'Vgstar_prime': Vgstar_prime, \
              'Lg_prime': Lg_prime, 'Lgstar_prime': Lgstar_prime, \
              'Ve_prime': Ve_prime, 'Vestar_prime': Vestar_prime, \
              'delta_Le': delta_Le, 'delta_Lestar': delta_Lestar, \
              'leakage1': leakage1, 'leakage2': leakage2,'leakage3': leakage3, \
              'leakage3': leakage3, 'chg_extraction': chg_extraction, 'chg_production': chg_production, \
              'chg_consumption': chg_consumption,'chg_Qeworld':chg_Qeworld, 'pai_g': pai_g, \
              'subsidy_ratio': subsidy_ratio, 'welfare':welfare, 'welfare_noexternality':welfare_noexternality}))

        minuswelfare.welfare = delta_U/Vg;  
        ob=-minuswelfare.welfare;
        return ob
        
        
    def equilibrium_hatalgebra(self, pe,*data):
        tb_mat, te, varphi, tax_scenario, ParaList, df = data
        alpha, theta, sigma, sigmastar, epsilonD,epsilonDstar, epsilonS,epsilonSstar, beta, gamma, logit = ParaList

        ## compute extraction tax, and jbar's
        te, jxbar_hat, jmbar_hat, j0_hat, tb_mat = computejbar(ParaList, pe, te, varphi, tb_mat, tax_scenario, df)

        jxbar_prime = jxbar_hat * df['jxbar']
        jmbar_prime = jmbar_hat * df['jmbar']
        j0_prime = j0_hat * df['jxbar']

        #if te is too large, HoGe stop producing
        petbte = pe + tb_mat[0] - te
        z = pe + tb_mat[0] >= te
        petbte = petbte * z

        ## compute home and foreign extractions
        Qe_prime, Qestar_prime = compute_qe(petbte, epsilonS, epsilonSstar, logit, beta, gamma, pe, df)

        ## compute consumptions around the world
        CeHH_prime, CeFH1_prime, CeFH2_prime, CeFH_prime, CeHF_prime, CeFF_prime, CeFH_hat, CeFH1_hat, CeHF_hat = compute_ce(pe, tb_mat,j0_hat, j0_prime, jmbar_hat, jmbar_prime,jxbar_hat, jxbar_prime, ParaList, df, tax_scenario)
        diff = Qe_prime + Qestar_prime - (CeHH_prime + CeFH_prime + CeHF_prime + CeFF_prime)
        #print('diff='+str(diff))
        return diff
    
    def computejbar(ParaList, pe, te, varphi, tb_mat, tax_scenario, df):

        alpha, theta, sigma, sigmastar, epsilonD,epsilonDstar, epsilonS,epsilonSstar, beta, gamma, logit = ParaList

        ## optimal values
        # jxbar_hat =   (1 - df['jxbar']) ** (-1) / (((1 - df['jxbar']) ** (-1) - 1) + (1 + (1 - alpha) * tb_mat[0]/pe) ** (-theta) * (1 + tb_mat[0]/pe) ** ((1 - alpha) * theta));
        jxbar_hat = pe**(-alpha*theta) * (pe+tb_mat[0])**(-(1-alpha)*theta) / ( df['jxbar'] * pe**(-alpha*theta) * (pe+tb_mat[0])**(-(1-alpha)*theta) + (1-df['jxbar']) * (pe + (1-alpha) * tb_mat[0])**(-theta));
        j0_hat = (pe+tb_mat[0])**(-(1-alpha)*theta) / (df['jxbar'] * (pe+tb_mat[0])**(-(1-alpha)*theta) + (1-df['jxbar']) * pe**(-(1-alpha)*theta) );
        jmbar_hat = 1

        if tax_scenario['tax_sce']=='Unilateral':
            te=varphi;
            tb_mat[1]=1;

        if tax_scenario['tax_sce']=='purete':
            jxbar_hat = 1;   
            jmbar_hat = 1;

        if tax_scenario['tax_sce']=='puretc':
            te=tb_mat[0];
            jxbar_hat = 1;
            jmbar_hat = 1;
            tb_mat[1]=1;

        if tax_scenario['tax_sce']=='puretp':
            te=tb_mat[0];
            jxbar_hat =  (1 - df['jxbar']) ** (-1) / ((1 - df['jxbar']) ** (-1) - 1 + (1 + tb_mat[0]/pe) ** (theta * (1 - alpha)));
            jmbar_hat = (1 + ((1 - df['jmbar']) ** (-1) - 1) ** (-1)) / (1 + ((1 - df['jmbar']) ** (-1) - 1) ** (-1) * (1 + tb_mat[0]/pe) ** (theta * (1 - alpha)));
            tb_mat[1]=1;

        if tax_scenario['tax_sce']=='EC_hybrid':
            te=varphi;
            jxbar_hat = 1;
            jmbar_hat = 1;
            tb_mat[1]=1;

        if tax_scenario['tax_sce']=='PC_hybrid':
            te=tb_mat[0];
            if tb_mat[1]<0:
                tb_mat[1]=0
            jxbar_hat =  (1 - df['jxbar']) ** (-1) / ((1 - df['jxbar']) ** (-1) - 1 + (1 + (tb_mat[0] - tb_mat[2] * tb_mat[0])/pe) ** (theta * (1 - alpha)));
            jmbar_hat = (1 + ((1 - df['jmbar']) ** (-1) - 1) ** (-1)) / (1 + ((1 - df['jmbar']) ** (-1) - 1) ** (-1) * ((pe + tb_mat[0])/(pe + tb_mat[1] * tb_mat[0])) ** (theta * (1 - alpha)));

        if tax_scenario['tax_sce']=='EP_hybrid':
            te=tb_mat[1];
            jxbar_hat =  (1 - df['jxbar']) ** (-1) / ((1 - df['jxbar']) ** (-1) - 1 + (1 + tb_mat[0]/pe) ** (theta * (1 - alpha)));
            jmbar_hat = (1 + ((1 - df['jmbar']) ** (-1) - 1) ** (-1)) / (1 + ((1 - df['jmbar']) ** (-1) - 1) ** (-1) * (1 + tb_mat[0]/pe) ** (theta * (1 - alpha)));

        if tax_scenario['tax_sce']=='EPC_hybrid':
            te=varphi;
            if tb_mat[1]<0:
                tb_mat[1]=0
            jxbar_hat =  (1 - df['jxbar']) ** (-1) / ((1 - df['jxbar']) ** (-1) - 1 + (1 + (tb_mat[0] - tb_mat[2] * tb_mat[0])/pe) ** (theta * (1 - alpha)));
            jmbar_hat = (1 + ((1 - df['jmbar']) ** (-1) - 1) ** (-1)) / (1 + ((1 - df['jmbar']) ** (-1) - 1) ** (-1) * ((pe + tb_mat[0])/(pe + tb_mat[1] * tb_mat[0])) ** (theta * (1 - alpha)))

        return (te, jxbar_hat, jmbar_hat, j0_hat, tb_mat)

    ## compute values for the incomplete beta functions
    def imcomp_betas(j0_prime, jxbar_prime, theta, sigmastar):
        def tempFunction(i, theta, sigmastar):
            return (i ** ((1 + theta) / theta - 1) * (1 - i) ** ((theta - sigmastar) / theta - 1)) 

        Bfunvec1_prime = quad(tempFunction,0,j0_prime, args=(theta, sigmastar))[0]
        Bfunvec2_prime = quad(tempFunction,0,jxbar_prime, args=(theta, sigmastar))[0]

        return (Bfunvec1_prime, Bfunvec2_prime)

    ## computes extraction values (home, foreign)
    def compute_qe(petbte, epsilonS, epsilonSstar, logit, beta, gamma, pe, df):
        if logit==1:
            epsilonS=beta*(1-gamma)/(1-gamma+gamma*petbte**beta)
            epsilonSstar=beta*(1-gamma)/(1-gamma+gamma*pe**beta)
            Qe_hat = (petbte)**beta/(1-gamma+gamma*(petbte)**beta)
            Qestar_hat = pe**beta/(1-gamma+gamma*pe**beta)

        ## compute hat values    
        Qe_hat = (petbte) ** epsilonS
        Qestar_hat = pe ** epsilonSstar

        ## compute final values
        Qe_prime = df['Qe'] * Qe_hat
        Qestar_prime = df['Qestar'] * Qestar_hat

        return (Qe_prime, Qestar_prime)


    ## computes consumption values (home, import, export, foreign)
    def compute_ce(pe, tb_mat,j0_hat, j0_prime, jmbar_hat, jmbar_prime,jxbar_hat, jxbar_prime, ParaList, df, tax_scenario):
        alpha, theta, sigma, sigmastar, epsilonD,epsilonDstar, epsilonS,epsilonSstar, beta, gamma, logit = ParaList

        Bfunvec1_prime, Bfunvec2_prime = imcomp_betas(j0_prime, jxbar_prime, theta, sigmastar)

        CeHH_hat = (pe + tb_mat[0]) ** (-epsilonD) * jmbar_hat ** (1 + (1 - sigma)/theta);
        CeHH_prime = df['CeHH'] * CeHH_hat;


        # CeFH_hat = (1 + (1 - sigmastar)/theta) * pe ** (-(1 - alpha) * sigmastar) * (pe + tb_mat[0]) ** (-alpha) * Bfunvec_prime/(df['jxbar'] ** (1 +1/theta)) * (1 - df['jxbar']) ** (sigmastar/theta);
        CeFH1_hat = (pe +tb_mat[0])**(-epsilonDstar) * j0_hat**(1 + (1 - sigmastar)/theta);
        CeFH2_hat = (1 + (1 - sigmastar)/theta) * ((1-df['jxbar'])/df['jxbar'])**(sigmastar/theta) * pe**(-epsilonDstar) * (1 + tb_mat[0]/pe)**(-alpha) * (Bfunvec2_prime - Bfunvec1_prime)/df['jxbar']**(1+(1-sigmastar)/theta);
        CeFH1_prime = df['CeFH'] * CeFH1_hat;
        CeFH2_prime = df['CeFH'] * CeFH2_hat;
        CeFH_hat = CeFH1_hat + CeFH2_hat;
        if tax_scenario['Base']==1:
            CeFH_hat = pe ** (-epsilonDstar) * jxbar_hat ** (1 + (1 - sigmastar)/theta);

        if tax_scenario['tax_sce']=='puretp' or tax_scenario['tax_sce']=='EP_hybrid':
            CeFH_hat = (pe + tb_mat[0]) ** (-epsilonDstar) * jxbar_hat ** (1 + (1 - sigmastar)/theta);

        if tax_scenario['tax_sce']=='PC_hybrid' or tax_scenario['tax_sce']=='EPC_hybrid':
            CeFH_hat = (pe + tb_mat[0] - tb_mat[2] * tb_mat[0]) ** (-epsilonDstar) * jxbar_hat ** (1 + (1 - sigmastar)/theta);

        if np.isnan(CeFH_hat)==True:
            CeFH_hat=0
        CeFH_prime =df['CeFH'] * CeFH_hat;


        CeHF_hat = (pe + tb_mat[0]) ** (-epsilonD);
        if tax_scenario['tax_sce']=='puretp' or tax_scenario['tax_sce']=='EP_hybrid':
            CeHF_hat = (pe) ** (-epsilonD) * ((1 - jmbar_prime)/(1 - df['jmbar'])) ** (1 + (1 - sigma)/theta);

        if tax_scenario['tax_sce']=='PC_hybrid' or tax_scenario['tax_sce']=='EPC_hybrid':
            CeHF_hat = (pe + tb_mat[1] * tb_mat[0]) ** (-epsilonD) * ((1 - jmbar_prime)/(1 - df['jmbar'])) ** (1 + (1 - sigma)/theta);

        CeHF_prime = df['CeHF'] * CeHF_hat;


        CeFF_prime = df['CeFF'] * ((1 - jxbar_prime)/(1-df['jxbar'])) ** (1 + (1 - sigmastar)/theta) * pe ** (-epsilonDstar);

        return (CeHH_prime, CeFH1_prime, CeFH2_prime, CeFH_prime, CeHF_prime, CeFF_prime, CeFH_hat, CeFH1_hat, CeHF_hat)



    def comp_vg(pe, tb_mat, CeFH1_hat, j0_prime, jxbar_prime, CeFH_hat, CeHF_hat, df, tax_scenario, ParaList):

        alpha, theta, sigma, sigmastar, epsilonD,epsilonDstar, epsilonS,epsilonSstar, beta, gamma, logit = ParaList

        ##
        VgHH = df['CeHH']/(1 - alpha)
        VgFF = df['CeFF']/(1 - alpha)

        VgFH = df['CeFH'] /(1 - alpha)
        # VgFH_prime = VgFH * pe ** ((1 - sigmastar) * (1 - alpha)) * (1 - (1 - jxbar_prime) ** (1 + (1 - sigmastar)/theta))/ (df['jxbar'] * (1 - df['jxbar']) ** ( (1-sigmastar)/theta))
        VgFH1_hat = (pe + tb_mat[0]) * CeFH1_hat
        VgFH2_hat = pe**(1 - epsilonDstar) * ((1-j0_prime)**(1+(1-sigmastar)/theta) - (1-jxbar_prime)**(1+(1-sigmastar)/theta))/ (df['jxbar']  * (1 - df['jxbar'] )**( (1-sigmastar)/theta))
        VgFH1_prime = VgFH * VgFH1_hat
        VgFH2_prime = VgFH * VgFH2_hat
        VgFH_hat = VgFH1_hat + VgFH2_hat

        if tax_scenario['Base']==1:
            VgFH_hat = pe * CeFH_hat

        if tax_scenario['tax_sce']=='puretp' or tax_scenario['tax_sce']=='EP_hybrid':
            VgFH_hat = (pe + tb_mat[0]) * CeFH_hat

        if tax_scenario['tax_sce']=='PC_hybrid' or tax_scenario['tax_sce']=='EPC_hybrid':
            VgFH_hat = (pe + tb_mat[0] - tb_mat[2] * tb_mat[0]) * CeFH_hat

        if np.isnan(VgFH_hat)==True:
                VgFH_hat=0
        VgFH_prime = VgFH * VgFH_hat

        VgHF = df['CeHF']/(1 - alpha)
        VgHF_hat = (pe + alpha * tb_mat[0]) * CeHF_hat

        if tax_scenario['tax_sce']=='puretp' or tax_scenario['tax_sce']=='EP_hybrid':
            VgHF_hat = pe * CeHF_hat

        if tax_scenario['tax_sce']=='PC_hybrid' or tax_scenario['tax_sce']=='EPC_hybrid':
            VgHF_hat = (pe + alpha * tb_mat[1] * tb_mat[0]) * CeHF_hat

        VgHF_prime = VgHF * VgHF_hat

        return (VgHH, VgFF, VgFH1_prime, VgFH2_prime, VgFH_prime, VgHF_prime, VgFH, VgHF)

    def comp_ve(pe, tb_mat, Ce_prime, tax_scenario, CeHH_prime, CeHF_prime, CeFH_prime, CeFH1_prime, CeFH2_prime, CeFF_prime, Cestar_prime):
        Ve_prime=(pe+tb_mat[0]) * Ce_prime
        if tax_scenario['tax_sce']=='puretp' or tax_scenario['tax_sce']=='EP_hybrid':
            Ve_prime = (pe+tb_mat[0]) * CeHH_prime + pe * CeHF_prime

        if tax_scenario['tax_sce']=='PC_hybrid' or tax_scenario['tax_sce']=='EPC_hybrid':
            Ve_prime = (pe+tb_mat[0]) * CeHH_prime + (pe + tb_mat[1]*tb_mat[0]) * CeHF_prime

        Vestar_prime= (pe+tb_mat[0]) * CeFH_prime  + pe * CeFF_prime
        Vestar_prime= (pe+tb_mat[0]) * CeFH1_prime + pe * CeFH2_prime + pe * CeFF_prime

        if tax_scenario['tax_sce']=='puretc' or tax_scenario['tax_sce']=='EC_hybrid' :
            Vestar_prime = pe * Cestar_prime

        if tax_scenario['tax_sce']=='PC_hybrid' or tax_scenario['tax_sce']=='EPC_hybrid':
            Vestar_prime = (pe + tb_mat[0] - tb_mat[2]*tb_mat[0]) * CeFH_prime + pe * CeFF_prime

        return (Ve_prime, Vestar_prime)

    def comp_vgfin(pe, tb_mat, Ce_hat, Cestar_prime, CeHH_prime, CeHF_prime, CeFH_prime, CeFF_prime,VgFH_prime, ParaList, df, tax_scenario):
        alpha, theta, sigma, sigmastar, epsilonD,epsilonDstar, epsilonS,epsilonSstar, beta, gamma, logit = ParaList

        Vg = df['Ce'] /(1-alpha)
        Vg_prime_hat = (pe + tb_mat[0]) * Ce_hat
        Vg_prime = Vg * Vg_prime_hat
        if tax_scenario['tax_sce']=='puretp' or tax_scenario['tax_sce']=='EP_hybrid':
            Vg_prime = CeHH_prime/(1-alpha) * (pe + tb_mat[0]) + CeHF_prime/(1-alpha) * pe

        if tax_scenario['tax_sce']=='PC_hybrid' or tax_scenario['tax_sce']=='EPC_hybrid':
            Vg_prime = CeHH_prime/(1-alpha) * (pe + tb_mat[0]) + CeHF_prime/(1-alpha) * (pe + tb_mat[1] * tb_mat[0]);   

        Vgstar = df['Cestar'] /(1-alpha)
        Vgstar_prime = VgFH_prime + CeFF_prime/(1-alpha)* pe
        if tax_scenario['tax_sce']=='puretc' or tax_scenario['tax_sce']=='EC_hybrid' :
            Vgstar_prime = Cestar_prime/(1-alpha)* pe

        if tax_scenario['tax_sce']=='PC_hybrid' or tax_scenario['tax_sce']=='EPC_hybrid':
            Vgstar_prime = CeFF_prime/(1-alpha)* pe + CeFH_prime/(1-alpha)* (pe + tb_mat[0] - tb_mat[2]*tb_mat[0])

        return (Vg, Vg_prime, Vgstar, Vgstar_prime)

    def comp_lg(pe, tb_mat, VgFH_prime, Ce_hat, Ge_prime, Gestar_prime, CeHH_prime, CeFH_prime, CeHF_prime, CeFF_prime, ParaList, df, tax_scenario):
        alpha, theta, sigma, sigmastar, epsilonD,epsilonDstar, epsilonS,epsilonSstar, beta, gamma, logit = ParaList

        Lg = alpha/(1-alpha) * df['Ge']
        Lg_prime = alpha/(1-alpha) * (pe + tb_mat[0]) * Ge_prime
        if tax_scenario['tax_sce']=='puretc' or tax_scenario['tax_sce']=='EC_hybrid':
            Lg_prime = alpha/(1-alpha) * (pe + tb_mat[0]) * CeHH_prime + alpha/(1-alpha) * pe * CeFH_prime

        if tax_scenario['tax_sce']=='PC_hybrid' or tax_scenario['tax_sce']=='EPC_hybrid':
            Lg_prime = alpha/(1-alpha) * (pe + tb_mat[0]) * CeHH_prime + alpha/(1-alpha) * (pe + tb_mat[0] - tb_mat[2] * tb_mat[0]) * CeFH_prime    

        Lgstar = alpha/(1-alpha) * df['Gestar']
        Lgstar_prime = alpha/(1-alpha) * (pe+tb_mat[0]) * CeHF_prime +alpha/(1-alpha) * pe * CeFF_prime
        if tax_scenario['tax_sce']=='puretp' or tax_scenario['tax_sce']=='EP_hybrid':
            Lgstar_prime = alpha/(1-alpha) * pe * Gestar_prime
        if tax_scenario['tax_sce']=='PC_hybrid' or tax_scenario['tax_sce']=='EPC_hybrid':
            Lgstar_prime = alpha/(1-alpha) * (pe + tb_mat[1]*tb_mat[0]) * CeHF_prime + alpha/(1-alpha) * pe * CeFF_prime

        return (Lg, Lg_prime, Lgstar, Lgstar_prime)

    def comp_delta(Lg, Lg_prime, Lgstar, Lgstar_prime, Qeworld_prime, Vg, Vgstar, df, j0_prime, jmbar_prime, jxbar_prime, pe, petbte, tb_mat, tax_scenario, varphi, ParaList):

        alpha, theta, sigma, sigmastar, epsilonD,epsilonDstar, epsilonS,epsilonSstar, beta, gamma, logit = ParaList

        if pe < 0:
            pe = 0.0001

        delta_Le = (epsilonS/(epsilonS + 1)) * df['Qe'] * (petbte**(epsilonS + 1) - 1);
        delta_Lestar = (epsilonSstar/(epsilonSstar + 1)) * df['Qestar'] * (pe**(epsilonSstar + 1) - 1);

        def Func(a, beta, gamma):
            return (((1-gamma)*a**beta)/(1-gamma+gamma*a**beta)**2)

        if logit==1:
            delta_Le = beta * df['Qe'] * quad(Func,1,petbte, args=(beta, gamma))[0];
            delta_Lestar = beta * df['Qestar'] * quad(Func,1,pe, args=(beta, gamma))[0];

        delta_U = -delta_Le - delta_Lestar - (Lg_prime - Lg) - (Lgstar_prime - Lgstar) \
               + Vg * (alpha - 1) * math.log(pe + tb_mat[0]) + Vgstar * (1/theta) * math.log(df['jxbar']/j0_prime * (pe+tb_mat[0])**(-(1-alpha)*theta)) \
               - varphi * (Qeworld_prime - df['Qeworld']);

        if tax_scenario['tax_sce']=='puretc' or tax_scenario['tax_sce']=='purete' or tax_scenario['tax_sce']=='EC_hybrid':
            delta_U = -delta_Le - delta_Lestar - (Lg_prime - Lg) - (Lgstar_prime - Lgstar) \
               + Vg * (alpha - 1) * math.log(pe +  tb_mat[0]) + Vgstar * (alpha - 1) * math.log(pe) \
               - varphi * (Qeworld_prime - df['Qeworld']);

        if tax_scenario['tax_sce']=='puretp' or tax_scenario['tax_sce']=='EP_hybrid':
            delta_U = -delta_Le - delta_Lestar - (Lg_prime - Lg) - (Lgstar_prime - Lgstar) \
                   +  Vg * ((alpha - 1) * math.log(pe + tb_mat[0]) + 1/theta * math.log(df['jmbar']/jmbar_prime)) \
                   +  Vgstar * ((alpha - 1) * math.log(pe + tb_mat[0]) + 1/theta * math.log(df['jxbar']/jxbar_prime)) \
                   - varphi * (Qeworld_prime - df['Qeworld']);

        if tax_scenario['tax_sce']=='PC_hybrid' or tax_scenario['tax_sce']=='EPC_hybrid':
            delta_U = -delta_Le - delta_Lestar - (Lg_prime - Lg) - (Lgstar_prime - Lgstar) \
                   +  Vg * ((alpha - 1) * math.log(pe + tb_mat[0]) + 1/theta * math.log(df['jmbar']/jmbar_prime)) \
                   +  Vgstar * ((alpha - 1) * math.log(pe + tb_mat[0] - tb_mat[2] * tb_mat[0]) + 1/theta * math.log(df['jxbar']/jxbar_prime)) \
                   - varphi * (Qeworld_prime - df['Qeworld']);
        return delta_Le, delta_Lestar, delta_U, pe

    def comp_leak(Qestar_prime, Gestar_prime, Cestar_prime, Qeworld_prime, df):

        leakage1 = -(Qestar_prime - df['Qestar'])/(Qeworld_prime - df['Qeworld']);
        leakage2 = -(Gestar_prime - df['Gestar'])/(Qeworld_prime - df['Qeworld']);
        leakage3 = -(Cestar_prime - df['Cestar'])/(Qeworld_prime - df['Qeworld']);   

        return (leakage1, leakage2, leakage3)
    
    def retrive(self):
        #print(self.results)
        ret = self.results
        ret['tb'] = self.tb
        ret['prop'] = self.prop
        ret['te'] = self.te
        return self.results

In [19]:
import math
import numpy as np
import pandas as pd
from scipy.optimize import minimize, least_squares
from scipy.optimize import fsolve
from scipy.integrate import quad
import matplotlib.pyplot as plt

In [534]:
argtemp = pd.Series({'first':1, 'second':2})
argtemp['first'] = 2
argtemp

first     2
second    2
dtype: int64

In [20]:
## scenario switch
case=3;  # 2 means no trade in goods; 3 means trade in both energy and goods
logit = 0; # 1 means logit estimations of supply elasticity; 0 means fixed elasticities at 0.5

In [21]:
## parameter values
alpha = 0.85;           # labor share parameter in manufacturing
theta = 4;              # scopevec for comparative advantage
sigma = 1;      # elasticity of demand for each individual manufactured good j at Home
sigmastar = 1;  # elasticity of demand for each individual manufactured good j at Foreign
epsilonD = alpha + (1 - alpha) * sigma;  #Home's elasticity of demand for embodied energy
epsilonDstar = alpha + (1 - alpha) * sigmastar;  #Foreign's elasticity of demand for embodied energy
# beta = 2.274853;
# gamma= 0.784877595;
beta=1.892412;
gamma=0.807998928;
epsilonS = 0.5;  #Homes's energy supply elasticity: beta/(1 - beta)
epsilonSstar = 0.5;  #Foreign's energy supply elasticity: betastar/(1 - betastar)

In [9]:
ParaList = (alpha, theta, sigma, sigmastar, epsilonD,epsilonDstar, epsilonS,epsilonSstar, beta, gamma, logit)

In [366]:
## import BAU values (seven regional scenarios in the order of US, EU, OECD, World, China, OECD plus China)
if case==2:
    df = pd.read_csv("../output/BaselineCarbon_2015_noTradeinGoods.csv",index_col=['region_scenario','regionbase'],header='infer')
elif case==3:
    df = pd.read_csv("../output/BaselineCarbon_2015.csv", index_col=['region_scenario','regionbase'],header='infer')
df['jxbar']=df['CeFH']/(df['CeFH'] + df['CeFF']);
df['jmbar']=df['CeHH']/(df['CeHH'] + df['CeHF']);

## choose which regional scenario to run (runs all if not executed)
df=df.drop([2,3,4,5,6,7,8,9,10])  
df

Qe     Qestar   Qeworld      CeHH      CeHF  \
region_scenario regionbase                                                      
1               US as Home  4.480045  27.795946  32.27599  4.598403  1.196111   

                                CeFH       CeFF        Ce     Cestar  \
region_scenario regionbase                                             
1               US as Home  0.421601  26.059874  5.794514  26.481474   

                                  Ge     Gestar   Ceworld   Geworld     jxbar  \
region_scenario regionbase                                                      
1               US as Home  5.020004  27.255985  32.27599  32.27599  0.015921   

                               jmbar  
region_scenario regionbase            
1               US as Home  0.793579

In [513]:
tax_scenario= pd.DataFrame({'tax_sce': ['Unilateral','purete','puretc','puretp','EC_hybrid','EP_hybrid','PC_hybrid','EPC_hybrid'], 'Base':[0,1,1,1,1,1,1,1]},index=[2, 3, 4, 5, 6, 7, 8, 9])
#df = pd.DataFrame({'Qe': 4.4800448, 'Qestar': 27.795946, 'Qeworld': 32.27599, 'CeHH': 4.598403, 'CeHF': 1.196111,'CeFH': 0.42160103,'CeFF': 26.059874,'Ce': 5.7945137,'Cestar': 26.481474,'Ge': 5.0200038,'Gestar': 27.255985,'Ceworld': 32.27599,'Geworld': 32.27599,'jxbar': 0.015920602214279298,'jmbar':  0.7935787194577492}, index = [1])

In [550]:
def temp(tax_scenario, alpha, theta, sigma, sigmastar, epsilonD,epsilonDstar, epsilonS,epsilonSstar, beta, gamma, logit):
    ParaList = (alpha, theta, sigma, sigmastar, epsilonD,epsilonDstar, epsilonS,epsilonSstar, beta, gamma, logit)
    te = 0; #initial value of extraction tax for iteration
    tb_mat = [0, 1];  #initial value of  border adjustment and proportion of it (prop is mainly used for PC hybrid)
    varphi = 2
    tax_df=df.apply(mintemp, axis=1, raw=False, args=(tb_mat, te, varphi, tax_scenario, ParaList))
    varphilist = np.arange (0,5,0.2) # marginal damages

    # use for quick test: varphilist = [2] or varphilist = np.arange (1.7,2.5,0.1)
    #varphilist = [2]
    output=[]
    for varphi in varphilist:    
        te = 0; #initial value of extraction tax for iteration
        tb_mat = [0, 1]  #initial value of  border adjustment and proportion of it (prop is mainly used for PC hybrid)

        ## calculate for optimal taxes by maximizing welfare
        tax_df=df.apply(mintemp, axis=1, raw=False, args=(tb_mat, te, varphi, tax_scenario, ParaList))
        
        output.append(tax_df)
        #print(varphi)

    output = pd.concat(output, axis=0, join='outer',  ignore_index=False, keys=None, levels=None, names=None, verify_integrity=False,copy=True)
    output.reset_index(level=0, inplace=True)
    output = output.sort_values(by=['region_scenario','varphi'])
    if tax_scenario['tax_sce']=='purete' or tax_scenario['tax_sce']=='EP_hybrid':
        output.te[output.Qe_prime==0]=output.pe+output.tb
    #print(tax_df)
    
    # appended_df = pd.merge(df, tax_df, on=['region_scenario','regionbase'])
    # output_df = appended_df.apply(callback, axis=1, raw=False, result_type=None, args=(varphi, tax_scenario, ParaList))
    # output_df = pd.merge(tax_df, output_df, on=['region_scenario','regionbase'])
    # output = [tax_df]
    # output = pd.concat(output, axis=0, join='outer',  ignore_index=False, keys=None, levels=None, names=None, verify_integrity=False,copy=True)
    # output.reset_index(level=0, inplace=True)
    # output = output.sort_values(by=['region_scenario','varphi'])
    #print(output)
    return tax_df

In [551]:
def mintemp(df, tb_mat, te, varphi, tax_scenario, ParaList):
    tax_temp = taxrow(df, tax_scenario, varphi, ParaList)
    tax_temp.opt_tax()
    ret = tax_temp.retrive()
    #print(ret)
    return ret

In [552]:
temp_df = tax_scenario.apply(temp, axis=1, args=(alpha, theta, sigma, sigmastar, epsilonD,epsilonDstar, epsilonS,epsilonSstar, beta, gamma, logit))
#temp_df.head()

C:\Users\Alex\AppData\Roaming\Python\Python39\site-packages\scipy\optimize\_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


In [553]:
output_list=[]
for i in range(2,len(tax_scenario)+1):
    output_list.append(temp_df.loc[i])
Outcomes = pd.concat(output_list, axis=0, join='outer', ignore_index=False, keys=tax_scenario['tax_sce'], levels=None, verify_integrity=False,copy=True)
Outcomes.reset_index(level=0, inplace=True)
Outcomes

tax_sce  varphi        pe  jxbar_prime  \
region_scenario regionbase                                              
1               US as Home  Unilateral     4.8  1.012278     0.021496   
                US as Home      purete     4.8  1.104719     0.015921   
                US as Home      puretc     4.8  0.921111     0.015921   
                US as Home      puretp     4.8  0.951548     0.010199   
                US as Home   EC_hybrid     4.8  1.020783     0.015921   
                US as Home   EP_hybrid     4.8  1.052713     0.010434   
                US as Home   PC_hybrid     4.8  0.918263     0.011695   

                            jmbar_prime  j0_prime                 Qe_prime  \
region_scenario regionbase                                                   
1               US as Home     0.793579  0.008887                      0.0   
                US as Home     0.793579  0.015921  [0.0013766865418962364]   
                US as Home     0.793579  0.008602                 4.299702   
                US as Home     0.710033  0.010199                 4.370164   
                US as Home     0.793579  0.008932                      0.0   
                US as Home     0.714738  0.010434                      0.0   
                US as Home     0.793525  0.008586                 4.293049   

                            Qestar_prime      Qeworld_prime  CeHH_prime  ...  \
region_scenario regionbase                                               ...   
1               US as Home     27.966064          27.966064    1.698812  ...   
                US as Home     29.215098  [29.216474215142]    4.162510  ...   
                US as Home     26.677027          30.976729    1.767484  ...   
                US as Home     27.114202          31.484365    2.038717  ...   
                US as Home     28.083305          28.083305    1.699049  ...   
                US as Home     28.519140           28.51914    1.927380  ...   
                US as Home     26.635748          30.928797    1.767076  ...   

                            chg_production  chg_consumption  \
region_scenario regionbase                                    
1               US as Home       -1.216168        -0.656107   
                US as Home       -2.583658        -2.510239   
                US as Home        1.495550         2.268022   
                US as Home        2.055827         1.198404   
                US as Home       -1.284745        -0.539164   
                US as Home       -0.792809        -1.459898   
                US as Home        1.705463         2.220201   

                                  chg_Qeworld     pai_g  subsidy_ratio  \
region_scenario regionbase                                               
1               US as Home          -4.309926 -4.797355       0.200704   
                US as Home  [-3.059515784858] -0.294330       0.000000   
                US as Home          -1.299261 -4.501759       0.214869   
                US as Home          -0.791625 -2.861497       0.143928   
                US as Home          -4.192685 -4.796295       0.198527   
                US as Home           -3.75685 -3.228912       0.135083   
                US as Home          -1.347193 -4.502951       0.215721   

                                        welfare  welfare_noexternality  \
region_scenario regionbase                                               
1               US as Home             39.70861             -13.844573   
                US as Home  [29.69650842885437]     [-8.3196475273868]   
                US as Home             10.66472              -5.479313   
                US as Home             6.826859              -3.009509   
                US as Home            38.865839             -13.230557   
                US as Home            35.610909             -11.070005   
                US as Home            11.096449              -5.643161   

                                  tb        pro

In [451]:
df_new = tax_scenario.apply(temp, axis=1, args=(alpha, theta, sigma, sigmastar, epsilonD,epsilonDstar, epsilonS,epsilonSstar, beta, gamma, logit))
#df_new.head()

In [322]:
def min_new(df, tb_mat, te, varphi, tax_scenario, ParaList):
    alpha, theta, sigma, sigmastar, epsilonD,epsilonDstar, epsilonS, epsilonSstar, beta, gamma, logit = ParaList

    if tax_scenario['tax_sce']=='Baseline':
        tb=0
        prop=1
        te=0
        return pd.Series({'tb': tb, 'prop': prop, 'te': te})
    elif tax_scenario['tax_sce']=='PC_hybrid':
        res = minimize(objective_fun, [1, 0, 1, 1], bounds=[(0,np.inf),(0,None),(0,1),(0, 1)],method='trust-constr',args=(te, varphi, tax_scenario, ParaList, df));
        tb_mat = res.x;
        tb=tb_mat[1];
        prop=tb_mat[2];   
        prop2=tb_mat[3];
        te=tb
        return pd.Series({'tb': tb, 'prop': prop, 'prop2': prop2,'te': te})
    
    elif tax_scenario['tax_sce']=='EPC_hybrid':
        res = minimize(objective_fun, [1, 0, 1, 1], bounds=[(0,np.inf),(0,None),(0,1),(0, 1)],method='trust-constr',args=(te, varphi, tax_scenario, ParaList, df));
        tb_mat = res.x;
        tb=tb_mat[1];
        prop=tb_mat[2];   
        prop2=tb_mat[3];
        te=varphi
        return pd.Series({'tb': tb, 'prop': prop, 'prop2': prop2,'te': te})
  
    elif tax_scenario['tax_sce']=='EP_hybrid':
        res = minimize(objective_fun, [1, 0, 1], bounds=[(0,np.inf),(0,None),(0,None)],method='L-BFGS-B', args=(te, varphi, tax_scenario, ParaList, df));
        tb_mat = res.x;
        tb=tb_mat[1];
        te=tb_mat[2];   
        prop=te-tb;
        return pd.Series({'tb': tb, 'prop': prop, 'te': te})
    elif tax_scenario['tax_sce']=='Unilateral' or tax_scenario['tax_sce']=='puretc' or tax_scenario['tax_sce']=='puretp' or tax_scenario['tax_sce']=='EC_hybrid' :
        res = minimize(objective_fun, [1.04557906, 0.7, 1], method='nelder-mead', args=(te, varphi, tax_scenario, ParaList, df), bounds = [(0,np.inf), (0, np.inf), (0,1)], options = {'maxfev': 100000}, tol = 0.00001)
        print(res.fun)
        tb_mat = res.x;
        pe = tb_mat[0]
        tb=tb_mat[1]
        prop=tb_mat[2]
        #print("tax returned is", tb_mat, "flag is", res.success, "tax scenario is", tax_scenario['tax_sce'])
        if tax_scenario['tax_sce']=='puretc' or tax_scenario['tax_sce']=='puretp':
            te=tb
        elif tax_scenario['tax_sce']=='Unilateral' or tax_scenario['tax_sce']=='EC_hybrid':
            te=varphi
        return pd.Series({'pe': pe, 'tb': tb, 'prop': prop, 'te': te})
    elif tax_scenario['tax_sce']=='purete':
        res = minimize(objective_fun, [1,1,1], method='nelder-mead', args=(varphi, te, tax_scenario, ParaList, df),  bounds = [(0,np.inf), (0, np.inf), (0,1)], options = {'maxfev': 100000}, tol = 0.00001);
        te = res.x[0]
        tb_mat = [0,1];
        tb=tb_mat[0];
        prop=tb_mat[1];  
        return pd.Series({'tb': tb, 'prop': prop, 'te': te})

In [359]:
def objective_fun(p, te, varphi, tax_scenario, ParaList, df):
     #solve for equilibrium
    pe = p[0]
    tb_mat = p[1:]
    te = varphi
    
    alpha, theta, sigma, sigmastar, epsilonD,epsilonDstar, epsilonS, epsilonSstar, beta, gamma, logit = ParaList
    
    ## compute extraction tax, and jbar_hat's
    te, jxbar_hat, jmbar_hat, j0_hat, tb_mat = computejbar(ParaList, pe, te, varphi, tb_mat, tax_scenario, df)
    
    jxbar_prime = jxbar_hat * df['jxbar'];
    jmbar_prime = jmbar_hat * df['jmbar'];
    j0_prime = j0_hat * df['jxbar'];

    #if te is too large, Home stop producing
    petbte = pe + tb_mat[0] - te;
    z = pe + tb_mat[0] >= te;
    petbte = petbte * z;
    
    ## home and foreign extractions
    Qe_prime, Qestar_prime = compute_qe(petbte, epsilonS, epsilonSstar, logit, beta, gamma, pe, df)
    
    ## compute consumptions around the world
    CeHH_prime, CeFH1_prime, CeFH2_prime, CeFH_prime, CeHF_prime, CeFF_prime, CeFH_hat, CeFH1_hat, CeHF_hat = compute_ce(pe, tb_mat,j0_hat, j0_prime, jmbar_hat, jmbar_prime,jxbar_hat, jxbar_prime, ParaList, df, tax_scenario)
    
    ## compute value of exports and imports
    VgHH, VgFF, VgFH1_prime, VgFH2_prime, VgFH_prime, VgHF_prime, VgFH, VgHF = comp_vg(pe, tb_mat, CeFH1_hat, j0_prime, jxbar_prime, CeFH_hat, CeHF_hat, df, tax_scenario, ParaList)
    
    Ge_prime = CeHH_prime + CeFH_prime;
    Gestar_prime = CeFF_prime + CeHF_prime;
    Ce_prime = CeHH_prime + CeHF_prime;
    Ce_hat = Ce_prime/df['Ce'];
    Cestar_prime = CeFF_prime + CeFH_prime;
    Qeworld_prime=Qe_prime+Qestar_prime;
    pai_g = VgFH - (pe + tb_mat[0]) * df['CeFH'] / (1 - alpha);
    subsidy_ratio = (tb_mat[0]/pe * (1 - alpha)) / (1 + tb_mat[0]/pe * (1 - alpha));

    leakage1, leakage2, leakage3 = comp_leak(Qestar_prime, Gestar_prime, Cestar_prime, Qeworld_prime, df)
    
    # compute objective
    diff = Qe_prime + Qestar_prime - (CeHH_prime + CeFH_prime + CeHF_prime + CeFF_prime)
    
    Ceworld_prime = Ce_prime + Cestar_prime
    epsilonSw = (Qe_prime) * epsilonS / Qeworld_prime + Qestar_prime * epsilonSstar / Qeworld_prime
    epsilonDw = Ce_prime * epsilonD / Ceworld_prime + Cestar_prime * epsilonDstar / Ceworld_prime
    epsilonG = CeHH_prime * epsilonD / Ge_prime + CeFH_prime * epsilonDstar / Ge_prime
    epsilonGstar = CeHF_prime * epsilonD / Gestar_prime + CeFF_prime * epsilonDstar / Gestar_prime
    epsilonGw = epsilonDw
    
    re = (pe + tb_mat[0]) / pe
    tejxj = (1+ (1- sigmastar)/theta) * df['CeFH'] * jxbar_hat**((1-sigmastar)/theta) * (pe * re)**(-epsilonDstar) / df['jxbar']
    ejyj = (1+ (1- sigmastar)/theta) * df['CeHH'] * jmbar_hat**((1-sigma) / theta) * (pe * re)**(-epsilonD) / df['jmbar']
    djxdre = -(theta * (1-alpha)) * jxbar_prime * (1-jxbar_prime) / re
    djmdre = -(theta * (1-alpha)) * jmbar_prime * (1-jmbar_prime) / re
    #djxdre = (alpha - 1) * (df['jxbar'] - 1) * df['jxbar'] * theta * re**((1-alpha) * theta -1) / ((df['jxbar'] - 1) * re**((1-alpha) * theta) - df['jxbar'])**2
    #djmdre = (alpha - 1) * (df['jmbar'] - 1) * df['jmbar'] * theta * re**((1-alpha) * theta -1) / ((df['jmbar'] - 1) * re**((1-alpha) * theta) - df['jmbar'])**2
    leaknum = re * (ejyj * djmdre + tejxj * djxdre)
    leakdenum = ejyj * djmdre + tejxj * djxdre - epsilonD * CeHH_prime / re - epsilonDstar * CeFH_prime / re
    leak = leaknum / leakdenum
    
    leaknum2 = re * tejxj * djxdre
    leakdenum2 = tejxj * djxdre - epsilonDstar * CeFH_prime / re
    leak2 = leaknum2 / leakdenum2
    
    if tax_scenario['tax_sce']=='Unilateral':
        S = (pe + tb_mat[0]) * CeFH2_prime / (1 - alpha) - VgFH2_prime
        numerator = varphi * epsilonSstar * Qestar_prime - sigmastar * (1-alpha) * S
        denominator = epsilonSstar * Qestar_prime + epsilonDstar * CeFF_prime
        diff1 = tb_mat[0] * denominator - numerator
        
    if tax_scenario['tax_sce']=='purete':
        numerator = varphi * epsilonSstar * Qestar_prime
        denominator = epsilonSstar * Qestar_prime + epsilonDw * Ceworld_prime
        diff1 = tb_mat[0] * denominator - numerator

    if tax_scenario['tax_sce']=='puretc':
        numerator = varphi * epsilonSw * Qeworld_prime
        denominator = epsilonSw * Qeworld_prime + epsilonDstar * Cestar_prime
        diff1 = tb_mat[0] * denominator - numerator
    
    if tax_scenario['tax_sce']=='puretp':
        numerator = varphi * epsilonSw * Qeworld_prime
        denominator = epsilonSw * Qeworld_prime + epsilonGstar * Gestar_prime + leak * epsilonG * Ge_prime
        diff1 = tb_mat[0] * denominator - numerator

    
    if tax_scenario['tax_sce']=='EC_hybrid':
        numerator = varphi * epsilonSstar * Qestar_prime
        denominator = epsilonSstar * Qestar_prime + epsilonDstar * Cestar_prime
        diff1 = tb_mat[0] * denominator - numerator
    
    if tax_scenario['tax_sce']=='PC_hybrid':
        numerator = varphi * epsilonSw * Qeworld_prime
        denominator = epsilonSw * Qeworld_prime + epsilonDstar * CeFF_prime + leak2 * epsilonDstar * CeFH_prime
        diff1 = tb_mat[0] * denominator - numerator

    if tax_scenario['tax_sce']=='EP_hybrid':
        numerator = varphi * epsilonSstar * Qestar_prime
        denominator = epsilonSstar * Qestar_prime + epsilonGstar * Gestar_prime + leak * epsilonG * Ge_prime
        diff1 = tb_mat[0] * denominator - numerator

    if tax_scenario['tax_sce']=='EPC_hybrid':
        numerator = varphi * epsilonSstar * Qestar_prime - sigmastar * (1-alpha) * pai_g
        denominator = epsilonSstar * Qestar_prime + epsilonDstar * CeFF_prime
        diff1 = tb_mat[0] * denominator - numerator
    #print(diff1, diff)
    return abs(diff) + abs(diff1)

In [338]:
df_new = tax_scenario.apply(temp, axis=1, args=(alpha, theta, sigma, sigmastar, epsilonD,epsilonDstar, epsilonS,epsilonSstar, beta, gamma, logit))

   tb  prop  te
1   0     1   0
         tb       prop   te
1  0.726671 -71.768109  2.0
    tb  prop       te
1  0.0   1.0  1.10475
         tb      prop        te
1  0.719738 -70.99884  0.719738
         tb       prop        te
1  0.533347 -52.359686  0.533347
         tb       prop   te
1  0.721912 -71.059979  2.0
         tb      prop        te
1  0.540729  1.099767  1.640496


C:\Users\Alex\AppData\Roaming\Python\Python39\site-packages\scipy\optimize\_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


       tb      prop     prop2      te
1  0.7222  0.998526  0.510942  0.7222
         tb      prop    prop2   te
1  0.724764  0.996767  0.50062  2.0


In [243]:
def temp2(tax_scenario, alpha, theta, sigma, sigmastar, epsilonD,epsilonDstar, epsilonS,epsilonSstar, beta, gamma, logit):
    ParaList = (alpha, theta, sigma, sigmastar, epsilonD,epsilonDstar, epsilonS,epsilonSstar, beta, gamma, logit)
    te = 0; #initial value of extraction tax for iteration
    tb_mat = [0, 1];  #initial value of  border adjustment and proportion of it (prop is mainly used for PC hybrid)
    varphi = 2
    tax_df=df.apply(min_new, axis=1, raw=False, args=(tb_mat, te, varphi, tax_scenario, ParaList))
    print(tax_scenario['tax_sce'])
    print(tax_df)
    return tax_df

In [304]:
objective_fun([1.04557906, 0.72667065, 100], 2, 2, {'tax_sce': 'Unilateral', 'Base': 0}, ParaList, {'Qe': 4.4800448, 'Qestar': 27.795946, 'Qeworld': 32.27599, 'CeHH': 4.598403, 'CeHF': 1.196111,'CeFH': 0.42160103,'CeFF': 26.059874,'Ce': 5.7945137,'Cestar': 26.481474,'Ge': 5.0200038,'Gestar': 27.255985,'Ceworld': 32.27599,'Geworld': 32.27599,'jxbar': 0.015920602214279298,'jmbar':  0.7935787194577492})

2.543402253252225e-07